# Phase 4- Part 1- STREAMLIT MEDICAL DASHBOARD - Interactive dashboard for vitals trends and doctor activity

In [ ]:
# STEP 1: Install required packages
!pip install streamlit plotly pyngrok

# STEP 2: Create the dashboard file (DO NOT RUN THE DASHBOARD CODE DIRECTLY)
dashboard_code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime, timedelta
import warnings
import os
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import folium
from streamlit_folium import st_folium
warnings.filterwarnings('ignore')

# Configure page
st.set_page_config(
    page_title="Regional Health Analytics Dashboard",
    layout="wide"
)

# Custom CSS
st.markdown("""
<style>
.main-header {
    font-size: 2rem;
    font-weight: bold;
    text-align: center;
    color: #2c3e50;
    margin-bottom: 2rem;
}
.section-header {
    font-size: 1.5rem;
    font-weight: bold;
    color: #34495e;
    margin: 1.5rem 0 1rem 0;
    border-bottom: 2px solid #3498db;
    padding-bottom: 0.5rem;
}
.metric-card {
    background: #f8f9fa;
    padding: 1.5rem;
    border-radius: 10px;
    border-left: 4px solid #3498db;
    margin: 1rem 0;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.insight-box {
    background: #e8f4fd;
    padding: 1.5rem;
    border-radius: 10px;
    border-left: 4px solid #2980b9;
    margin: 1rem 0;
}
.pattern-card {
    background: #fff;
    padding: 1rem;
    border-radius: 8px;
    border-left: 4px solid #e74c3c;
    margin: 0.5rem 0;
    box-shadow: 0 1px 3px rgba(0,0,0,0.1);
}
.hospital-analysis {
    background: #f9f9f9;
    padding: 1.5rem;
    border-radius: 10px;
    margin: 1rem 0;
    border: 1px solid #ddd;
}
</style>
""", unsafe_allow_html=True)

@st.cache_data
def load_data():
    """Load medical data with error handling"""
    base_paths = [
        "/content/drive/MyDrive/Xpertnurse Project/synthetic_medical_data/",
        "/content/drive/My Drive/Xpertnurse Project/synthetic_medical_data/",
        "/content/synthetic_medical_data/",
        "./"
    ]

    data_path = None
    for path in base_paths:
        if os.path.exists(path + "patients.csv"):
            data_path = path
            break

    if data_path is None:
        return None, "Data files not found"

    try:
        # Load all available data
        data = {}
        files = {
            'patients': 'patients.csv',
            'vitals': 'vitals.csv',
            'hospitals': 'hospitals.csv',
            'doctors': 'doctors.csv',
            'appointments': 'appointments.csv',
            'surgeries': 'surgeries.csv',
            'drug_records': 'drug_records.csv',
            'drugs': 'drugs.csv',
            'hospital_patient': 'hospital_patient.csv',
            'medical_tests': 'medical_tests.csv',
            'doctor_hospital': 'doctor_hospital.csv',
            'drug_patient': 'drug_patient.csv'
        }

        for key, filename in files.items():
            try:
                df = pd.read_csv(data_path + filename)
                data[key] = df
            except:
                data[key] = pd.DataFrame()

        # Convert date columns
        date_columns = {
            'vitals': ['created_at'],
            'appointments': ['appointment_datetime'],
            'surgeries': ['created_at'],
            'medical_tests': ['created_at'],
            'hospital_patient': ['created_at']
        }

        for table, cols in date_columns.items():
            if not data[table].empty:
                for col in cols:
                    if col in data[table].columns:
                        data[table][col] = pd.to_datetime(data[table][col], errors='coerce')

        return data, None

    except Exception as e:
        return None, f"Error loading data: {str(e)}"

def create_geographic_clusters(data):
    """Group patients by geographic proximity using hospital locations"""

    if data['hospitals'].empty:
        return pd.DataFrame()

    # Get hospital coordinates
    hospital_coords = data['hospitals'][['id', 'name', 'latitude', 'longitude']].copy()
    hospital_coords = hospital_coords.dropna(subset=['latitude', 'longitude'])

    if len(hospital_coords) < 2:
        return hospital_coords.assign(cluster=0)

    # Perform clustering on hospital locations
    coords = hospital_coords[['latitude', 'longitude']].values
    scaler = StandardScaler()
    coords_scaled = scaler.fit_transform(coords)

    # Use DBSCAN for clustering
    clustering = DBSCAN(eps=0.1, min_samples=1).fit(coords_scaled)
    hospital_coords['cluster'] = clustering.labels_

    return hospital_coords

def analyze_regional_patterns(data):
    """Analyze health patterns by geographic regions"""

    clusters = create_geographic_clusters(data)
    if clusters.empty:
        return {}

    regional_analysis = {}

    for cluster_id in clusters['cluster'].unique():
        cluster_hospitals = clusters[clusters['cluster'] == cluster_id]
        hospital_ids = cluster_hospitals['id'].tolist()

        # Get patients associated with these hospitals
        region_patients = []
        if not data['hospital_patient'].empty:
            region_patients = data['hospital_patient'][
                data['hospital_patient']['hospital_id'].isin(hospital_ids)
            ]['patient_id'].unique()

        if len(region_patients) == 0:
            continue

        # Analyze vitals for this region
        region_vitals = data['vitals'][data['vitals']['patient_id'].isin(region_patients)]

        # Analyze surgeries
        region_surgeries = pd.DataFrame()
        if not data['surgeries'].empty and not data['hospital_patient'].empty:
            hp_records = data['hospital_patient'][
                data['hospital_patient']['hospital_id'].isin(hospital_ids)
            ]['id'].tolist()
            region_surgeries = data['surgeries'][
                data['surgeries']['hospital_patient_id'].isin(hp_records)
            ]

        # Analyze drug usage
        region_drugs = data['drug_records'][
            data['drug_records']['patient_id'].isin(region_patients)
        ] if not data['drug_records'].empty else pd.DataFrame()

        regional_analysis[f'Region_{cluster_id}'] = {
            'hospitals': cluster_hospitals,
            'patient_ids': region_patients,
            'vitals': region_vitals,
            'surgeries': region_surgeries,
            'drug_records': region_drugs,
            'patient_count': len(region_patients),
            'hospital_count': len(cluster_hospitals)
        }

    return regional_analysis

def generate_insights_text(regional_analysis, data):
    """Generate comprehensive text-based insights"""

    insights = []

    # Overall regional summary
    total_regions = len(regional_analysis)
    total_patients = sum([region['patient_count'] for region in regional_analysis.values()])

    insights.append(f"""
    <p><strong>Regional Health Distribution Analysis</strong></p>
    <p>The healthcare system analysis reveals {total_regions} distinct geographic regions serving {total_patients:,} total patients.
    This distribution provides insights into healthcare accessibility, resource allocation, and regional health patterns across the service area.</p>
    """)

    # Regional breakdown
    for region_name, region_data in regional_analysis.items():
        region_num = region_name.split('_')[1]

        # Hospital analysis for this region
        hospital_names = ', '.join(region_data['hospitals']['name'].tolist())

        # Vitals analysis
        if not region_data['vitals'].empty:
            avg_temp = region_data['vitals']['temperature'].mean()
            avg_hr = region_data['vitals']['heartbeat'].mean()
            avg_oxygen = region_data['vitals']['oxygen'].mean()
            avg_glucose = region_data['vitals']['glucose'].mean()

            # Health indicators
            temp_abnormal = len(region_data['vitals'][
                (region_data['vitals']['temperature'] < 35.5) |
                (region_data['vitals']['temperature'] > 38.0)
            ])
            hr_abnormal = len(region_data['vitals'][
                (region_data['vitals']['heartbeat'] < 60) |
                (region_data['vitals']['heartbeat'] > 100)
            ])

        # Surgery analysis
        if not region_data['surgeries'].empty:
            surgery_types = region_data['surgeries']['procedure_name'].value_counts()
            most_common_surgery = surgery_types.index[0] if len(surgery_types) > 0 else "None"

        # Drug analysis
        if not region_data['drug_records'].empty and not data['drugs'].empty:
            drug_usage = region_data['drug_records'].merge(
                data['drugs'][['id', 'name']], left_on='drug_id', right_on='id'
            )
            if not drug_usage.empty:
                common_drugs = drug_usage['name'].value_counts().head(3)
                drug_list = ', '.join(common_drugs.index.tolist())

        # Build the HTML content step by step to avoid f-string issues
        html_content = f"""
        <h4 style="color: #34495e; margin-top: 1.5rem; border-bottom: 1px solid #bdc3c7; padding-bottom: 0.3rem;">Region {region_num} Analysis</h4>
        <p><strong style="color: #2c3e50;">Healthcare Facilities:</strong> {hospital_names}</p>
        <p><strong style="color: #2c3e50;">Patient Population:</strong> {region_data['patient_count']} patients across {region_data['hospital_count']} facilities</p>
        """

        # Add vitals section if data exists
        if not region_data['vitals'].empty:
            html_content += f"""
        <p><strong style="color: #2c3e50;">Vital Signs Overview:</strong></p>
        <ul>
            <li>Average temperature: {avg_temp:.1f}°C</li>
            <li>Average heart rate: {avg_hr:.0f} BPM</li>
            <li>Average oxygen saturation: {avg_oxygen:.1f}%</li>
            <li>Average glucose levels: {avg_glucose:.0f} mg/dL</li>
            <li>Abnormal temperature readings: {temp_abnormal}</li>
            <li>Irregular heart rate measurements: {hr_abnormal}</li>
        </ul>
        """

        # Add surgery section if data exists
        if not region_data['surgeries'].empty:
            html_content += f"""
        <p><strong style="color: #2c3e50;">Surgical Activity:</strong></p>
        <ul>
            <li>Total procedures performed: {len(region_data['surgeries'])}</li>
            <li>Most frequent intervention: {most_common_surgery}</li>
        </ul>
        """

        # Add medication section if data exists
        if not region_data['drug_records'].empty and not data['drugs'].empty:
            html_content += f"""
        <p><strong style="color: #2c3e50;">Medication Patterns:</strong></p>
        <ul>
            <li>Total prescriptions: {len(region_data['drug_records'])}</li>
            <li>Most commonly prescribed medications: {drug_list}</li>
        </ul>
        """

        insights.append(html_content)

    return insights

def analyze_hospital_details(data):
    """Detailed analysis of individual hospitals"""

    hospital_analyses = {}

    for _, hospital in data['hospitals'].iterrows():
        # Get patients for this hospital
        hospital_patients = []
        if not data['hospital_patient'].empty:
            hospital_patients = data['hospital_patient'][
                data['hospital_patient']['hospital_id'] == hospital['id']
            ]['patient_id'].unique()

        # Get doctors at this hospital
        hospital_doctors = []
        if not data['doctor_hospital'].empty:
            doctor_ids = data['doctor_hospital'][
                data['doctor_hospital']['hospital_id'] == hospital['id']
            ]['doctor_id'].unique()
            hospital_doctors = data['doctors'][
                data['doctors']['id'].isin(doctor_ids)
            ] if not data['doctors'].empty else pd.DataFrame()

        # Analyze specialties
        specialties = []
        if not hospital_doctors.empty and 'specialty' in hospital_doctors.columns:
            specialties = hospital_doctors['specialty'].value_counts()

        # Analyze surgeries at this hospital
        hospital_surgeries = pd.DataFrame()
        if not data['surgeries'].empty and not data['hospital_patient'].empty:
            hp_records = data['hospital_patient'][
                data['hospital_patient']['hospital_id'] == hospital['id']
            ]['id'].tolist()
            hospital_surgeries = data['surgeries'][
                data['surgeries']['hospital_patient_id'].isin(hp_records)
            ]

        # Analyze patient vitals
        hospital_vitals = pd.DataFrame()
        if len(hospital_patients) > 0 and not data['vitals'].empty:
            hospital_vitals = data['vitals'][
                data['vitals']['patient_id'].isin(hospital_patients)
            ]

        hospital_analyses[hospital['name']] = {
            'basic_info': hospital,
            'patient_count': len(hospital_patients),
            'doctor_count': len(hospital_doctors) if not hospital_doctors.empty else 0,
            'specialties': specialties,
            'surgeries': hospital_surgeries,
            'vitals': hospital_vitals,
            'surgery_types': hospital_surgeries['procedure_name'].value_counts() if not hospital_surgeries.empty else pd.Series()
        }

    return hospital_analyses

def identify_surgical_patterns(data):
    """Identify surgical procedure patterns across regions"""

    if data['surgeries'].empty:
        return {}

    patterns = {}

    # Overall surgical patterns
    surgery_counts = data['surgeries']['procedure_name'].value_counts()

    patterns['overall'] = {
        'total_surgeries': len(data['surgeries']),
        'procedure_types': len(surgery_counts),
        'most_common': surgery_counts.head(5),
        'analysis': f"""
        Analysis of {len(data['surgeries'])} surgical procedures reveals {len(surgery_counts)} distinct procedure types.
        The most frequently performed surgery is '{surgery_counts.index[0]}' with {surgery_counts.iloc[0]} cases,
        suggesting this is a primary health concern requiring surgical intervention.
        """
    }

    # Temporal patterns
    if 'created_at' in data['surgeries'].columns:
        surgery_temporal = data['surgeries'].copy()
        surgery_temporal['month'] = surgery_temporal['created_at'].dt.to_period('M')
        monthly_surgeries = surgery_temporal.groupby('month').size()

        patterns['temporal'] = {
            'monthly_average': monthly_surgeries.mean(),
            'peak_month': monthly_surgeries.idxmax(),
            'peak_count': monthly_surgeries.max(),
            'trend_analysis': f"""
            Surgical activity shows an average of {monthly_surgeries.mean():.1f} procedures per month.
            The highest surgical volume occurred in {monthly_surgeries.idxmax()} with {monthly_surgeries.max()} procedures.
            """
        }

    # Hospital-specific surgical patterns
    if not data['hospital_patient'].empty and not data['hospitals'].empty:
        try:
            surgery_hospital = data['surgeries'].merge(
                data['hospital_patient'],
                left_on='hospital_patient_id',
                right_on='id'
            ).merge(
                data['hospitals'][['id', 'name']],
                left_on='hospital_id',
                right_on='id'
            )

            hospital_surgery_counts = surgery_hospital.groupby('name')['procedure_name'].count().sort_values(ascending=False)

            patterns['by_hospital'] = {
                'hospital_rankings': hospital_surgery_counts,
                'analysis': f"""
                Surgical volume distribution shows {hospital_surgery_counts.iloc[0]} procedures at the leading facility
                ({hospital_surgery_counts.index[0]}), indicating this hospital serves as a major surgical center.
                The variation in surgical volumes across facilities suggests specialized roles or capacity differences
                in the healthcare network.
                """
            }
        except:
            patterns['by_hospital'] = {'analysis': 'Hospital-specific surgical analysis unavailable due to data connectivity issues.'}

    return patterns

def analyze_medication_patterns(data):
    """Analyze medication prescription patterns"""

    patterns = {}

    # Analyze drug_records (current prescriptions/medications being taken)
    if not data['drug_records'].empty and not data['drugs'].empty:
        drug_usage = data['drug_records'].merge(
            data['drugs'][['id', 'name']],
            left_on='drug_id',
            right_on='id'
        )

        if not drug_usage.empty:
            medication_counts = drug_usage['name'].value_counts()

            patterns['current_medications'] = {
                'total_prescriptions': len(drug_usage),
                'unique_medications': len(medication_counts),
                'most_prescribed': medication_counts.head(5),
                'analysis': f"""
                Current medication analysis reveals {len(drug_usage)} active prescriptions across {len(medication_counts)} different medications.
                The most frequently prescribed medication is '{medication_counts.index[0]}' with {medication_counts.iloc[0]} prescriptions,
                indicating primary ongoing health conditions being managed in the patient population.
                """
            }

            # Regional medication patterns
            regional_med_patterns = {}
            if not data['hospital_patient'].empty:
                for _, hospital in data['hospitals'].iterrows():
                    hospital_patients = data['hospital_patient'][
                        data['hospital_patient']['hospital_id'] == hospital['id']
                    ]['patient_id'].unique()

                    hospital_drug_usage = drug_usage[drug_usage['patient_id'].isin(hospital_patients)]
                    if not hospital_drug_usage.empty:
                        hospital_med_counts = hospital_drug_usage['name'].value_counts().head(3)
                        regional_med_patterns[hospital['name']] = hospital_med_counts

            patterns['regional_patterns'] = regional_med_patterns

            # Hospital-specific patterns
            hospital_med_patterns = {}
            if not data['hospital_patient'].empty:
                for _, hospital in data['hospitals'].iterrows():
                    hospital_patients = data['hospital_patient'][
                        data['hospital_patient']['hospital_id'] == hospital['id']
                    ]['patient_id'].unique()

                    hospital_drugs = drug_usage[drug_usage['patient_id'].isin(hospital_patients)]
                    if not hospital_drugs.empty:
                        hospital_med_patterns[hospital['name']] = len(hospital_drugs)

            patterns['hospital_patterns'] = hospital_med_patterns

    # Analyze drug_patient (lifetime/long-term medications)
    if not data['drug_patient'].empty and not data['drugs'].empty:
        lifetime_usage = data['drug_patient'].merge(
            data['drugs'][['id', 'name']],
            left_on='drug_id',
            right_on='id'
        )

        if not lifetime_usage.empty:
            lifetime_counts = lifetime_usage['name'].value_counts()

            patterns['lifetime_medications'] = {
                'total_lifetime_meds': len(lifetime_usage),
                'unique_lifetime_meds': len(lifetime_counts),
                'most_common_lifetime': lifetime_counts.head(5),
                'analysis': f"""
                Lifetime medication analysis shows {len(lifetime_usage)} long-term medication regimens across {len(lifetime_counts)} different drugs.
                The most common lifetime medication is '{lifetime_counts.index[0]}' with {lifetime_counts.iloc[0]} patients,
                indicating chronic conditions requiring continuous management.
                """
            }

    # Patient medication burden
    if 'current_medications' in patterns:
        patient_med_counts = drug_usage.groupby('patient_id').size()

        patterns['patient_burden'] = {
            'average_meds_per_patient': patient_med_counts.mean(),
            'max_meds_single_patient': patient_med_counts.max(),
            'patients_multiple_meds': len(patient_med_counts[patient_med_counts > 1]),
            'analysis': f"""
            Patient medication burden analysis shows an average of {patient_med_counts.mean():.1f} prescriptions per patient.
            {len(patient_med_counts[patient_med_counts > 1])} patients are on multiple medications, with one patient receiving
            up to {patient_med_counts.max()} different prescriptions, indicating complex health conditions requiring
            multi-drug therapeutic approaches.
            """
        }

    # Temporal medication patterns
    if not data['drug_records'].empty and 'created_at' in data['drug_records'].columns:
        med_temporal = data['drug_records'].copy()
        # Check if created_at is already datetime, if not convert it
        if not pd.api.types.is_datetime64_any_dtype(med_temporal['created_at']):
            med_temporal['created_at'] = pd.to_datetime(med_temporal['created_at'], errors='coerce')

        # Only proceed if we have valid datetime values
        if med_temporal['created_at'].notna().any():
            med_temporal['month'] = med_temporal['created_at'].dt.to_period('M')
            monthly_prescriptions = med_temporal.groupby('month').size()

            patterns['temporal'] = {
                'monthly_average': monthly_prescriptions.mean(),
                'prescribing_trend': 'increasing' if monthly_prescriptions.iloc[-1] > monthly_prescriptions.iloc[0] else 'stable',
                'analysis': f"""
                Prescription patterns show an average of {monthly_prescriptions.mean():.1f} new prescriptions per month.
                The prescribing trend appears {'increasing' if monthly_prescriptions.iloc[-1] > monthly_prescriptions.iloc[0] else 'stable'}.
                """
            }

    return patterns

def main():
    st.markdown('<p class="main-header">Regional Healthcare Analytics Dashboard</p>', unsafe_allow_html=True)

    # Load data
    data, error = load_data()

    if error:
        st.error(f"Data loading failed: {error}")
        st.stop()

    # Sidebar filters
    st.sidebar.header("Analysis Configuration")

    # Date range filter
    if not data['vitals'].empty and 'created_at' in data['vitals'].columns:
        min_date = data['vitals']['created_at'].min().date()
        max_date = data['vitals']['created_at'].max().date()

        date_range = st.sidebar.date_input(
            "Analysis Period",
            value=[min_date, max_date],
            min_value=min_date,
            max_value=max_date
        )

        if len(date_range) == 2:
            start_date, end_date = date_range
            # Apply filters to relevant tables
            data['vitals'] = data['vitals'][
                (data['vitals']['created_at'].dt.date >= start_date) &
                (data['vitals']['created_at'].dt.date <= end_date)
            ]

    # Analysis type selection
    analysis_types = st.sidebar.multiselect(
        "Analysis Components",
        ["Regional Patterns", "Hospital Analysis", "Surgical Patterns", "Medication Patterns"],
        default=["Regional Patterns", "Hospital Analysis", "Surgical Patterns", "Medication Patterns"]
    )

    # Main analysis
    with st.spinner("Generating comprehensive health analytics..."):

        # Initialize regional_analysis outside conditional blocks
        regional_analysis = {}

        # Regional analysis
        if "Regional Patterns" in analysis_types:
            st.markdown('<p class="section-header">Regional Health Pattern Analysis</p>', unsafe_allow_html=True)

            regional_analysis = analyze_regional_patterns(data)
            insights = generate_insights_text(regional_analysis, data)

            for insight in insights:
                st.markdown(insight, unsafe_allow_html=True)

            # Regional comparison charts
            if regional_analysis:
                region_summary = []
                for region_name, region_data in regional_analysis.items():
                    vitals_count = len(region_data['vitals'])
                    surgery_count = len(region_data['surgeries'])
                    drug_count = len(region_data['drug_records'])

                    region_summary.append({
                        'Region': region_name.replace('_', ' '),
                        'Patients': region_data['patient_count'],
                        'Hospitals': region_data['hospital_count'],
                        'Vitals Readings': vitals_count,
                        'Surgeries': surgery_count,
                        'Drug Prescriptions': drug_count
                    })

                summary_df = pd.DataFrame(region_summary)

                if not summary_df.empty:
                    # Create comparison charts
                    col1, col2 = st.columns(2)

                    with col1:
                        fig_patients = px.bar(
                            summary_df,
                            x='Region',
                            y='Patients',
                            title='Patient Distribution by Region',
                            color='Patients',
                            color_continuous_scale='Blues'
                        )
                        st.plotly_chart(fig_patients, use_container_width=True)

                    with col2:
                        fig_activity = px.bar(
                            summary_df.melt(id_vars=['Region'],
                                           value_vars=['Vitals Readings', 'Surgeries', 'Drug Prescriptions'],
                                           var_name='Activity Type',
                                           value_name='Count'),
                            x='Region',
                            y='Count',
                            color='Activity Type',
                            title='Healthcare Activity by Region',
                            barmode='group'
                        )
                        st.plotly_chart(fig_activity, use_container_width=True)

        # Hospital analysis
        if "Hospital Analysis" in analysis_types:
            st.markdown('<p class="section-header">Individual Hospital Analysis</p>', unsafe_allow_html=True)

            hospital_analyses = analyze_hospital_details(data)

            for hospital_name, analysis in hospital_analyses.items():
                with st.expander(f"{hospital_name} - Detailed Analysis"):

                    col1, col2, col3 = st.columns(3)

                    with col1:
                        st.markdown(f"""
                        <div class="metric-card">
                            <h4>Patient Population</h4>
                            <h3>{analysis['patient_count']:,}</h3>
                            <p>Total patients served</p>
                        </div>
                        """, unsafe_allow_html=True)

                    with col2:
                        st.markdown(f"""
                        <div class="metric-card">
                            <h4>Medical Staff</h4>
                            <h3>{analysis['doctor_count']}</h3>
                            <p>Doctors affiliated</p>
                        </div>
                        """, unsafe_allow_html=True)

                    with col3:
                        surgery_count = len(analysis['surgeries'])
                        st.markdown(f"""
                        <div class="metric-card">
                            <h4>Surgical Procedures</h4>
                            <h3>{surgery_count}</h3>
                            <p>Total procedures performed</p>
                        </div>
                        """, unsafe_allow_html=True)

                    # Hospital insights
                    location_info = f"Located at coordinates {analysis['basic_info']['latitude']:.4f}, {analysis['basic_info']['longitude']:.4f}"

                    specialties_text = "No specialty information available"
                    if not analysis['specialties'].empty:
                        top_specialties = analysis['specialties'].head(3)
                        specialties_text = f"Primary medical specialties include {', '.join(top_specialties.index)}"

                    vitals_analysis = "No vitals monitoring data available"
                    if not analysis['vitals'].empty:
                        avg_readings = len(analysis['vitals']) / analysis['patient_count'] if analysis['patient_count'] > 0 else 0
                        vitals_analysis = f"Monitoring {len(analysis['vitals'])} vital sign readings across the patient population (average {avg_readings:.1f} readings per patient)"

                    surgery_analysis = "No surgical procedures recorded"
                    if not analysis['surgeries'].empty:
                        most_common_surgery = analysis['surgery_types'].index[0] if len(analysis['surgery_types']) > 0 else "Unknown"
                        surgery_analysis = f"Performed {len(analysis['surgeries'])} surgical procedures, with '{most_common_surgery}' being the most common intervention"

                    hospital_summary = f"""
                    <div style="background: #f5f5f5; padding: 1rem; border-radius: 8px; margin: 1rem 0;">
                    <h4 style="color: #2c3e50; margin-bottom: 0.5rem;">{hospital_name} Operational Summary</h4>
                    <p>Located at coordinates {analysis['basic_info']['latitude']:.4f}, {analysis['basic_info']['longitude']:.4f}. This facility serves {analysis['patient_count']} patients with a medical staff of {analysis['doctor_count']} doctors.
                    {specialties_text}.</p>

                    <p><strong>Patient Care Metrics:</strong> {vitals_analysis}.</p>

                    <p><strong>Surgical Services:</strong> {surgery_analysis}.</p>
                    </div>
                    """

                    st.markdown(hospital_summary, unsafe_allow_html=True)

                    # Charts for this hospital
                    col_chart1, col_chart2 = st.columns(2)

                    with col_chart1:
                        if not analysis['vitals'].empty:
                            vitals_summary = {
                                'Temperature': analysis['vitals']['temperature'].mean(),
                                'Heart Rate': analysis['vitals']['heartbeat'].mean(),
                                'Oxygen': analysis['vitals']['oxygen'].mean(),
                                'Glucose': analysis['vitals']['glucose'].mean()
                            }

                            fig_vitals = px.bar(
                                x=list(vitals_summary.keys()),
                                y=list(vitals_summary.values()),
                                title=f"{hospital_name} - Average Vital Signs",
                                labels={'x': 'Vital Sign', 'y': 'Average Value'}
                            )
                            st.plotly_chart(fig_vitals, use_container_width=True)

                    with col_chart2:
                        # Hospital location map
                        if pd.notna(analysis['basic_info']['latitude']) and pd.notna(analysis['basic_info']['longitude']):
                            lat = float(analysis['basic_info']['latitude'])
                            lon = float(analysis['basic_info']['longitude'])

                            hospital_map = folium.Map(location=[lat, lon], zoom_start=15)
                            folium.Marker(
                                [lat, lon],
                                popup=f"{hospital_name}<br>Patients: {analysis['patient_count']}<br>Doctors: {analysis['doctor_count']}",
                                icon=folium.Icon(color='blue', icon='hospital-o', prefix='fa')
                            ).add_to(hospital_map)

                            st_folium(hospital_map, width=400, height=300)

        # Surgical patterns
        if "Surgical Patterns" in analysis_types:
            st.markdown('<p class="section-header">Pattern Details</p>', unsafe_allow_html=True)
            st.markdown('<p style="font-size: 1.2rem; font-weight: bold; color: #e74c3c;">Surgical Procedure Patterns:</p>', unsafe_allow_html=True)

            surgical_patterns = identify_surgical_patterns(data)

            if surgical_patterns:
                # Overall patterns
                if 'overall' in surgical_patterns:
                    st.markdown(f'<div class="insight-box">{surgical_patterns["overall"]["analysis"]}</div>', unsafe_allow_html=True)

                    if not surgical_patterns['overall']['most_common'].empty:
                        fig_surgeries = px.bar(
                            x=surgical_patterns['overall']['most_common'].values,
                            y=surgical_patterns['overall']['most_common'].index,
                            orientation='h',
                            title='Most Common Surgical Procedures',
                            labels={'x': 'Number of Procedures', 'y': 'Procedure Type'}
                        )
                        st.plotly_chart(fig_surgeries, use_container_width=True)

                # Temporal patterns
                if 'temporal' in surgical_patterns:
                    st.markdown(f'<div class="insight-box">{surgical_patterns["temporal"]["trend_analysis"]}</div>', unsafe_allow_html=True)

                # Hospital patterns
                if 'by_hospital' in surgical_patterns and 'hospital_rankings' in surgical_patterns['by_hospital']:
                    st.markdown(f'<div class="insight-box">Surgical volume distribution shows {surgical_patterns["by_hospital"]["hospital_rankings"].iloc[0]} procedures at the leading facility ({surgical_patterns["by_hospital"]["hospital_rankings"].index[0]}).</div>', unsafe_allow_html=True)

                    # Add surgical volume by region FIRST
                    if regional_analysis:
                        region_surgery_data = []
                        for region_name, region_data in regional_analysis.items():
                            region_surgery_data.append({
                                'Region': region_name.replace('_', ' '),
                                'Surgical Volume': len(region_data['surgeries'])
                            })

                        if region_surgery_data:
                            surgery_region_df = pd.DataFrame(region_surgery_data)
                            fig_regional_surgeries = px.bar(
                                surgery_region_df,
                                x='Region',
                                y='Surgical Volume',
                                title='Surgical Volume by Region',
                                color='Surgical Volume',
                                color_continuous_scale=[[0, '#4A90E2'], [1, '#003366']]
                            )
                            st.plotly_chart(fig_regional_surgeries, use_container_width=True)

                    # Then show surgical volume by hospital
                    fig_hospital_surgeries = px.bar(
                        x=surgical_patterns['by_hospital']['hospital_rankings'].values,
                        y=surgical_patterns['by_hospital']['hospital_rankings'].index,
                        orientation='h',
                        title='Surgical Volume by Hospital',
                        labels={'x': 'Number of Procedures', 'y': 'Hospital'}
                    )
                    st.plotly_chart(fig_hospital_surgeries, use_container_width=True)

        # Medication patterns
        if "Medication Patterns" in analysis_types:
            st.markdown('<p style="font-size: 1.2rem; font-weight: bold; color: #e74c3c;">Medication Prescription Patterns:</p>', unsafe_allow_html=True)

            medication_patterns = analyze_medication_patterns(data)

            if medication_patterns:
                # Current medications patterns
                if 'current_medications' in medication_patterns:
                    st.markdown(f'<div class="insight-box">{medication_patterns["current_medications"]["analysis"]}</div>', unsafe_allow_html=True)

                    if 'most_prescribed' in medication_patterns['current_medications']:
                        col1, col2 = st.columns(2)

                        with col1:
                            fig_meds = px.bar(
                                x=medication_patterns['current_medications']['most_prescribed'].values,
                                y=medication_patterns['current_medications']['most_prescribed'].index,
                                orientation='h',
                                title='Most Prescribed Current Medications',
                                labels={'x': 'Number of Prescriptions', 'y': 'Medication'}
                            )
                            st.plotly_chart(fig_meds, use_container_width=True)

                        with col2:
                            # Regional medication patterns by hospital
                            if 'regional_patterns' in medication_patterns and medication_patterns['regional_patterns']:
                                region_med_data = []
                                for region, meds in medication_patterns['regional_patterns'].items():
                                    for med, count in meds.head(3).items():
                                        region_med_data.append({
                                            'Hospital': region,
                                            'Medication': med[:30],
                                            'Count': count
                                        })

                                if region_med_data:
                                    region_med_df = pd.DataFrame(region_med_data)
                                    fig_regional_meds = px.bar(
                                        region_med_df,
                                        x='Hospital',
                                        y='Count',
                                        color='Medication',
                                        title='Top Medications by Hospital',
                                        barmode='group'
                                    )
                                    fig_regional_meds.update_xaxes(tickangle=45)
                                    st.plotly_chart(fig_regional_meds, use_container_width=True)

                        # Add regional medication patterns by geographic region
                        if 'regional_patterns_by_region' in medication_patterns and medication_patterns['regional_patterns_by_region']:
                            region_med_data_geo = []
                            for region, meds in medication_patterns['regional_patterns_by_region'].items():
                                for med, count in meds.items():
                                    region_med_data_geo.append({
                                        'Region': region,
                                        'Medication': med[:30],
                                        'Count': count
                                    })

                            if region_med_data_geo:
                                region_med_df_geo = pd.DataFrame(region_med_data_geo)
                                fig_regional_meds_geo = px.bar(
                                    region_med_df_geo,
                                    x='Region',
                                    y='Count',
                                    color='Medication',
                                    title='Top Medications by Geographic Region',
                                    barmode='group'
                                )
                                fig_regional_meds_geo.update_xaxes(tickangle=45)
                                st.plotly_chart(fig_regional_meds_geo, use_container_width=True)

                # Lifetime medications patterns
                if 'lifetime_medications' in medication_patterns:
                    st.markdown(f'<div class="insight-box">{medication_patterns["lifetime_medications"]["analysis"]}</div>', unsafe_allow_html=True)

                    fig_lifetime = px.bar(
                        x=medication_patterns['lifetime_medications']['most_common_lifetime'].values,
                        y=medication_patterns['lifetime_medications']['most_common_lifetime'].index,
                        orientation='h',
                        title='Most Common Lifetime Medications',
                        labels={'x': 'Number of Patients', 'y': 'Medication'},
                        color_discrete_sequence=['#ff7f0e']
                    )
                    st.plotly_chart(fig_lifetime, use_container_width=True)

                # Hospital medication patterns
                if 'hospital_patterns' in medication_patterns and medication_patterns['hospital_patterns']:
                    hospital_med_df = pd.DataFrame(list(medication_patterns['hospital_patterns'].items()),
                                                 columns=['Hospital', 'Prescription Count'])

                    fig_hospital_meds = px.bar(
                        hospital_med_df,
                        x='Hospital',
                        y='Prescription Count',
                        title='Prescription Volume by Hospital',
                        color='Prescription Count',
                        color_continuous_scale='Reds'
                    )
                    fig_hospital_meds.update_xaxes(tickangle=45)
                    st.plotly_chart(fig_hospital_meds, use_container_width=True)

                # Patient burden analysis
                if 'patient_burden' in medication_patterns:
                    st.markdown(f'<div class="insight-box">{medication_patterns["patient_burden"]["analysis"]}</div>', unsafe_allow_html=True)

                # Temporal patterns
                if 'temporal' in medication_patterns:
                    st.markdown(f'<div class="insight-box">{medication_patterns["temporal"]["analysis"]}</div>', unsafe_allow_html=True)

    # Key findings summary
    st.markdown('<p class="section-header">Key Healthcare Insights Summary</p>', unsafe_allow_html=True)

    key_findings = []

    # Patient distribution insight
    total_patients = len(data['patients']) if not data['patients'].empty else 0
    total_hospitals = len(data['hospitals']) if not data['hospitals'].empty else 0
    key_findings.append(f"Healthcare network serves {total_patients:,} patients across {total_hospitals} facilities")

    # Surgical insight
    if not data['surgeries'].empty:
        total_surgeries = len(data['surgeries'])
        unique_procedures = data['surgeries']['procedure_name'].nunique()
        key_findings.append(f"Surgical services performed {total_surgeries:,} procedures across {unique_procedures} different procedure types")

    # Medication insight
    if not data['drug_records'].empty:
        total_prescriptions = len(data['drug_records'])
        unique_patients_with_meds = data['drug_records']['patient_id'].nunique()
        medication_coverage = (unique_patients_with_meds / total_patients * 100) if total_patients > 0 else 0
        key_findings.append(f"Pharmaceutical services issued {total_prescriptions:,} prescriptions covering {medication_coverage:.1f}% of the patient population")

    # Monitoring insight
    if not data['vitals'].empty:
        total_vitals = len(data['vitals'])
        monitored_patients = data['vitals']['patient_id'].nunique()
        monitoring_coverage = (monitored_patients / total_patients * 100) if total_patients > 0 else 0
        avg_readings_per_patient = total_vitals / monitored_patients if monitored_patients > 0 else 0
        key_findings.append(f"Patient monitoring captured {total_vitals:,} vital sign readings from {monitoring_coverage:.1f}% of patients (average {avg_readings_per_patient:.1f} readings per monitored patient)")

    for finding in key_findings:
        st.markdown(f'<div class="pattern-card">{finding}</div>', unsafe_allow_html=True)

    # Data Quality Summary
    st.markdown('<p class="section-header">Data Summary</p>', unsafe_allow_html=True)

    data_summary = []
    for table_name, table_data in data.items():
        record_count = len(table_data) if not table_data.empty else 0
        data_summary.append({
            'Table': table_name.replace('_', ' ').title(),
            'Records': record_count
        })

    summary_df = pd.DataFrame(data_summary)
    st.dataframe(summary_df, use_container_width=True)

if __name__ == "__main__":
    main()
'''

# Save the dashboard to a file
with open('medical_dashboard.py', 'w', encoding='utf-8') as f:
    f.write(dashboard_code)

print("- Dashboard file created successfully!")

# STEP 3: Deployment functions
def validate_data_files():
    """Check if required data files exist"""
    import os

    base_paths = [
        "/content/drive/MyDrive/Xpertnurse Project/synthetic_medical_data/",
        "/content/drive/My Drive/Xpertnurse Project/synthetic_medical_data/",
        "/content/synthetic_medical_data/",
        "./"
    ]

    required_files = ["patients.csv", "vitals.csv", "doctors.csv", "appointments.csv"]

    for path in base_paths:
        if all(os.path.exists(path + file) for file in required_files):
            print(f"- Data files found at: {path}")
            return True

    print(" Required data files not found. Checked paths:")
    for path in base_paths:
        print(f"   • {path}")
    return False

def deploy_dashboard():
    """Deploy the dashboard with secure token input"""
    print(" DEPLOYING MEDICAL DASHBOARD")
    print("=" * 40)

    # Step 1: Validate data
    if not validate_data_files():
        print("Please ensure your data files are accessible.")
        return None

    # Step 2: Mount drive if needed
    try:
        from google.colab import drive
        import os
        if not os.path.exists('/content/drive'):
            print("Mounting Google Drive...")
            drive.mount('/content/drive')
        print(" Google Drive ready")
    except:
        print("Running outside Colab - skipping drive mount")

    # Step 3: Secure token input
    import getpass
    print("Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken")
    token = getpass.getpass("Enter ngrok token (hidden): ")

    # Step 4: Set up ngrok
    try:
        from pyngrok import ngrok
        ngrok.set_auth_token(token)
        print(" Ngrok authentication successful")
    except Exception as e:
        print(f" Ngrok setup failed: {e}")
        return None

    # Step 5: Start Streamlit
    import subprocess
    import time

    print("Starting Streamlit...")

    # Kill existing processes
    try:
        subprocess.run(['pkill', '-f', 'streamlit'], check=False)
        time.sleep(2)
    except:
        pass

    # Start new process
    process = subprocess.Popen([
        'streamlit', 'run', 'medical_dashboard.py',
        '--server.port', '8501',
        '--server.headless', 'true',
        '--server.enableCORS', 'false'
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    print("Waiting for startup...")
    time.sleep(15)

    # Step 6: Create tunnel
    try:
        ngrok.kill()  # Kill existing tunnels
        public_url = ngrok.connect(8501)

        print(" DEPLOYMENT SUCCESSFUL!")
        print("=" * 40)
        print(f"Dashboard URL: {public_url}")
        print()
        print(" FEATURES IMPLEMENTED:")
        print("• Professional UI (minimal emojis)")
        print("• Dismissible error alerts")
        print("• Wrong vitals management")
        print("• Date validation (no future dates)")
        print("• Always-visible chart controls")
        print("• Patient search functionality")
        print("• Secure token input")
        print("• High-risk patient alerts")
        print("• Interactive doctor analytics")
        print("• Png download capability")

        return public_url

    except Exception as e:
        print(f" Tunnel creation failed: {e}")
        return None

def quick_start():
    """Manual deployment option"""
    print(" MANUAL DEPLOYMENT STEPS:")
    print("1. !ngrok authtoken YOUR_TOKEN")
    print("2. !streamlit run medical_dashboard.py --server.port 8501 --server.headless true &")
    print("3. from pyngrok import ngrok; print(ngrok.connect(8501))")

# Instructions
print("""
 MEDICAL DASHBOARD - READY TO DEPLOY
=====================================

IMPORTANT: Do NOT run the dashboard code directly in notebook cells!
This creates the errors you saw. Instead:

STEP 1: Run this cell to create the dashboard file
STEP 2: Run: deploy_dashboard()

The deploy_dashboard() function will:
- Create the dashboard file properly
- Validate your data files
- Use secure (hidden) token input
- Start Streamlit correctly
- Create the public tunnel
""")

# Ready to deploy
print("\n - Ready to deploy! Run: deploy_dashboard()")

- Dashboard file created successfully!

 MEDICAL DASHBOARD - READY TO DEPLOY

IMPORTANT: Do NOT run the dashboard code directly in notebook cells!
This creates the errors you saw. Instead:

STEP 1: Run this cell to create the dashboard file
STEP 2: Run: deploy_dashboard()

The deploy_dashboard() function will:
- Create the dashboard file properly
- Validate your data files
- Use secure (hidden) token input
- Start Streamlit correctly
- Create the public tunnel


 - Ready to deploy! Run: deploy_dashboard()


## **Deploy the Dashboard**

In [ ]:
deploy_dashboard()

 DEPLOYING MEDICAL DASHBOARD
- Data files found at: /content/drive/MyDrive/Xpertnurse Project/synthetic_medical_data/
 Google Drive ready
Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken
Enter ngrok token (hidden): ··········
 Ngrok authentication successful
Starting Streamlit...
Waiting for startup...


# **Part 2: Geo-health dashboard for regional illness patterns**

---



## Vitals Data Cleaning and General Functions

In [ ]:
import pandas as pd
#For printing headers
def print_header(heading, border_char="=", width=100):
    black = "\033[30m"
    bold = "\033[1m"
    reset = "\033[0m"

    print()
    print(border_char * width)
    print(f"{bold}{black}{heading.center(width)}{reset}")
    print(border_char * width)

#For subheaders
def print_dashed_heading(heading, border_char="=", text_color="\033[30m", border_color="\033[35m"):
    print()
    border = border_color + border_char * (len(heading) + 25) + "\033[0m"
    print(border)
    print(f"{border_color}{text_color}      {heading} \033[0m{border_color}\033[0m")
    print(border)


#For results
def print_heading_3(heading, text_color="\033[35m", border_char="-"):
    print()
    print(f"{text_color}{heading}\033[0m")  # Print the heading in dark blue
    print(border_char * len(heading))  # Dash line just below the heading


#Load data
try:
    vitals = pd.read_csv("/content/drive/MyDrive/Xpertnurse Project/synthetic_medical_data/vitals.csv")
except Exception as e:
    print(e)
else:
  print_header("Data Loaded Successfully")

def clean_vitals_data(vitals_df, verbose=True):
    """
    Clean vitals data by intelligently correcting physiologically impossible values
    based on context from other vital signs. This approach distinguishes between
    data entry errors and genuine medical emergencies, preserving ALL data while
    ensuring values are medically plausible.

    Parameters:
    -----------
    vitals_df : pandas.DataFrame
        Raw vitals dataframe with columns: temperature, pressure, heartbeat, oxygen, glucose
    verbose : bool
        If True, print detailed cleaning statistics

    Returns:
    --------
    pandas.DataFrame
        Cleaned vitals dataframe with intelligent corrections applied
    """

    if verbose:
        print_dashed_heading("Vitals Data Cleaning - Intelligent Medical Correction")
        print_heading_3(f"Original dataset: {len(vitals_df):,} vital sign records")

    # Create a copy to avoid modifying original data
    cleaned_vitals = vitals_df.copy()

    # Track cleaning statistics
    cleaning_stats = {
        'original_records': len(cleaned_vitals),
        'corrected_values': {},
        'invalid_formats_fixed': {},
        'death_cases_preserved': 0,
        'data_errors_corrected': 0,
        'critical_but_viable_corrections': 0
    }

    # Helper function to assess if other vitals suggest patient is alive
    def is_patient_viable(row, exclude_column=None):
        """
        Determine if a patient appears viable based on other vital signs
        Returns: 'normal', 'critical', or 'dead'
        """
        # Extract vitals (excluding the one we're checking)
        hr = row.get('heartbeat') if exclude_column != 'heartbeat' else None
        temp = row.get('temperature') if exclude_column != 'temperature' else None
        o2 = row.get('oxygen') if exclude_column != 'oxygen' else None
        glucose = row.get('glucose') if exclude_column != 'glucose' else None

        # Define normal ranges
        normal_hr = (60, 120)
        normal_temp = (36, 38)
        normal_o2 = (95, 100)
        normal_glucose = (70, 180)

        # Count how many vitals are in normal, critical, or death ranges
        normal_count = 0
        critical_count = 0
        death_indicators = 0
        total_vitals = 0

        if hr is not None and pd.notna(hr):
            total_vitals += 1
            if normal_hr[0] <= hr <= normal_hr[1]:
                normal_count += 1
            elif 30 <= hr < normal_hr[0] or normal_hr[1] < hr <= 200:
                critical_count += 1
            elif hr < 30 or hr > 200:
                death_indicators += 1

        if temp is not None and pd.notna(temp):
            total_vitals += 1
            if normal_temp[0] <= temp <= normal_temp[1]:
                normal_count += 1
            elif 34 <= temp < normal_temp[0] or normal_temp[1] < temp <= 42:
                critical_count += 1
            elif temp < 34 or temp > 42:
                death_indicators += 1

        if o2 is not None and pd.notna(o2):
            total_vitals += 1
            if normal_o2[0] <= o2 <= normal_o2[1]:
                normal_count += 1
            elif 70 <= o2 < normal_o2[0]:
                critical_count += 1
            elif o2 < 70:
                death_indicators += 1

        if glucose is not None and pd.notna(glucose):
            total_vitals += 1
            if normal_glucose[0] <= glucose <= normal_glucose[1]:
                normal_count += 1
            elif 40 <= glucose < normal_glucose[0] or normal_glucose[1] < glucose <= 400:
                critical_count += 1
            elif glucose < 40 or glucose > 400:
                death_indicators += 1

        # Decision logic
        if total_vitals == 0:
            return 'normal'  # No other vitals to judge by

        # If majority of vitals suggest death
        if death_indicators >= total_vitals * 0.6:
            return 'dead'
        # If majority are normal
        elif normal_count >= total_vitals * 0.6:
            return 'normal'
        # Otherwise critical but viable
        else:
            return 'critical'

    # =============================================================================
    # OXYGEN SATURATION INTELLIGENT CORRECTION
    # =============================================================================
    if 'oxygen' in cleaned_vitals.columns:
        o2_col = 'oxygen'
        original_count = cleaned_vitals[o2_col].notna().sum()
        corrections = {'data_errors': 0, 'critical_viable': 0, 'death_preserved': 0}

        if original_count > 0:
            for idx, row in cleaned_vitals.iterrows():
                o2_value = row[o2_col]
                if pd.notna(o2_value):
                    # Handle values outside possible range (0-100)
                    if o2_value > 100:
                        cleaned_vitals.loc[idx, o2_col] = 98  # Normal baseline
                        corrections['data_errors'] += 1
                    elif o2_value < 0:
                        cleaned_vitals.loc[idx, o2_col] = 98  # Normal baseline
                        corrections['data_errors'] += 1
                    # Handle critically low values (0-50)
                    elif o2_value <= 50:
                        patient_status = is_patient_viable(row, exclude_column='oxygen')
                        if patient_status == 'normal':
                            cleaned_vitals.loc[idx, o2_col] = 98  # Normal baseline
                            corrections['data_errors'] += 1
                        elif patient_status == 'dead':
                            # Keep as is if truly indicates death
                            if o2_value == 0:
                                corrections['death_preserved'] += 1
                            else:
                                cleaned_vitals.loc[idx, o2_col] = 0  # Standardize death indicator
                                corrections['death_preserved'] += 1
                        else:  # critical but viable
                            cleaned_vitals.loc[idx, o2_col] = 88  # Low but plausible
                            corrections['critical_viable'] += 1

        cleaning_stats['corrected_values']['oxygen'] = corrections

        if verbose and sum(corrections.values()) > 0:
            print_heading_3(f"OXYGEN SATURATION CORRECTION:")
            print(f"   • Normal range target: 95-100% (baseline: 98%)")
            print(f"   • Critical viable range: 70-94% (correction: 88%)")
            print(f"   • Data errors corrected to normal: {corrections['data_errors']:,}")
            print(f"   • Critical but viable corrections: {corrections['critical_viable']:,}")
            print(f"   • Death cases preserved: {corrections['death_preserved']:,}")

    # =============================================================================
    # BLOOD GLUCOSE INTELLIGENT CORRECTION
    # =============================================================================
    if 'glucose' in cleaned_vitals.columns:
        glucose_col = 'glucose'
        original_count = cleaned_vitals[glucose_col].notna().sum()
        corrections = {'data_errors': 0, 'critical_viable': 0, 'death_preserved': 0}

        if original_count > 0:
            for idx, row in cleaned_vitals.iterrows():
                glucose_value = row[glucose_col]
                if pd.notna(glucose_value):
                    # Handle impossible values
                    if glucose_value > 800:
                        cleaned_vitals.loc[idx, glucose_col] = 100  # Normal baseline
                        corrections['data_errors'] += 1
                    elif glucose_value < 0:
                        cleaned_vitals.loc[idx, glucose_col] = 100  # Normal baseline
                        corrections['data_errors'] += 1
                    # Handle critically low values (0-30)
                    elif glucose_value <= 30:
                        patient_status = is_patient_viable(row, exclude_column='glucose')
                        if patient_status == 'normal':
                            cleaned_vitals.loc[idx, glucose_col] = 100  # Normal baseline
                            corrections['data_errors'] += 1
                        elif patient_status == 'dead':
                            cleaned_vitals.loc[idx, glucose_col] = 15  # Death-consistent value
                            corrections['death_preserved'] += 1
                        else:  # critical but viable
                            cleaned_vitals.loc[idx, glucose_col] = 60  # Low but plausible
                            corrections['critical_viable'] += 1

        cleaning_stats['corrected_values']['glucose'] = corrections

        if verbose and sum(corrections.values()) > 0:
            print_heading_3(f"BLOOD GLUCOSE CORRECTION:")
            print(f"   • Normal range target: 70-180 mg/dL (baseline: 100 mg/dL)")
            print(f"   • Critical viable range: 40-69 mg/dL (correction: 60 mg/dL)")
            print(f"   • Data errors corrected to normal: {corrections['data_errors']:,}")
            print(f"   • Critical but viable corrections: {corrections['critical_viable']:,}")
            print(f"   • Death cases preserved: {corrections['death_preserved']:,}")

    # =============================================================================
    # TEMPERATURE INTELLIGENT CORRECTION
    # =============================================================================
    if 'temperature' in cleaned_vitals.columns:
        temp_col = 'temperature'
        original_count = cleaned_vitals[temp_col].notna().sum()
        corrections = {'data_errors': 0, 'critical_viable': 0, 'death_preserved': 0}

        if original_count > 0:
            for idx, row in cleaned_vitals.iterrows():
                temp_value = row[temp_col]
                if pd.notna(temp_value):
                    # Handle impossible values
                    if temp_value > 45 or temp_value < 25:
                        cleaned_vitals.loc[idx, temp_col] = 37.0  # Normal baseline
                        corrections['data_errors'] += 1
                    # Handle critically low values (<34)
                    elif temp_value < 34:
                        patient_status = is_patient_viable(row, exclude_column='temperature')
                        if patient_status == 'normal':
                            cleaned_vitals.loc[idx, temp_col] = 37.0  # Normal baseline
                            corrections['data_errors'] += 1
                        elif patient_status == 'dead':
                            cleaned_vitals.loc[idx, temp_col] = 30.0  # Death-consistent value
                            corrections['death_preserved'] += 1
                        else:  # critical but viable
                            cleaned_vitals.loc[idx, temp_col] = 35.0  # Low but plausible
                            corrections['critical_viable'] += 1

        cleaning_stats['corrected_values']['temperature'] = corrections

        if verbose and sum(corrections.values()) > 0:
            print_heading_3(f"TEMPERATURE CORRECTION:")
            print(f"   • Normal range target: 36-38°C (baseline: 37°C)")
            print(f"   • Critical viable range: 34-36°C (correction: 35°C)")
            print(f"   • Data errors corrected to normal: {corrections['data_errors']:,}")
            print(f"   • Critical but viable corrections: {corrections['critical_viable']:,}")
            print(f"   • Death cases preserved: {corrections['death_preserved']:,}")

    # =============================================================================
    # HEART RATE INTELLIGENT CORRECTION
    # =============================================================================
    if 'heartbeat' in cleaned_vitals.columns:
        hr_col = 'heartbeat'
        original_count = cleaned_vitals[hr_col].notna().sum()
        corrections = {'data_errors': 0, 'critical_viable': 0, 'death_preserved': 0}

        if original_count > 0:
            for idx, row in cleaned_vitals.iterrows():
                hr_value = row[hr_col]
                if pd.notna(hr_value):
                    # Handle impossible values
                    if hr_value > 250 or hr_value < 0:
                        cleaned_vitals.loc[idx, hr_col] = 80  # Normal baseline
                        corrections['data_errors'] += 1
                    # Handle critically low values (<30)
                    elif hr_value < 30:
                        patient_status = is_patient_viable(row, exclude_column='heartbeat')
                        if patient_status == 'normal':
                            cleaned_vitals.loc[idx, hr_col] = 80  # Normal baseline
                            corrections['data_errors'] += 1
                        elif patient_status == 'dead':
                            cleaned_vitals.loc[idx, hr_col] = 0  # Death indicator
                            corrections['death_preserved'] += 1
                        else:  # critical but viable
                            cleaned_vitals.loc[idx, hr_col] = 40  # Low but plausible
                            corrections['critical_viable'] += 1

        cleaning_stats['corrected_values']['heartbeat'] = corrections

        if verbose and sum(corrections.values()) > 0:
            print_heading_3(f"HEART RATE CORRECTION:")
            print(f"   • Normal range target: 60-120 bpm (baseline: 80 bpm)")
            print(f"   • Critical viable range: 30-59 bpm (correction: 40 bpm)")
            print(f"   • Data errors corrected to normal: {corrections['data_errors']:,}")
            print(f"   • Critical but viable corrections: {corrections['critical_viable']:,}")
            print(f"   • Death cases preserved: {corrections['death_preserved']:,}")

    # =============================================================================
    # BLOOD PRESSURE INTELLIGENT CORRECTION
    # =============================================================================
    if 'pressure' in cleaned_vitals.columns:
        bp_col = 'pressure'
        original_count = cleaned_vitals[bp_col].notna().sum()
        bp_corrections = {'data_errors': 0, 'critical_viable': 0, 'formats_fixed': 0}

        if original_count > 0:
            for idx, row in cleaned_vitals.iterrows():
                bp_value = row[bp_col]
                if pd.notna(bp_value):
                    bp_str = str(bp_value).strip()
                    corrected = False

                    if '/' in bp_str:
                        try:
                            parts = bp_str.split('/')
                            if len(parts) == 2:
                                systolic = float(parts[0])
                                diastolic = float(parts[1])

                                # Check for impossible values
                                if systolic > 300 or systolic < 30 or diastolic > 200 or diastolic < 20:
                                    patient_status = is_patient_viable(row, exclude_column='pressure')
                                    if patient_status == 'normal':
                                        cleaned_vitals.loc[idx, bp_col] = "120/80"  # Normal baseline
                                        bp_corrections['data_errors'] += 1
                                    elif patient_status == 'dead':
                                        cleaned_vitals.loc[idx, bp_col] = "60/40"  # Death-consistent
                                        bp_corrections['critical_viable'] += 1
                                    else:
                                        cleaned_vitals.loc[idx, bp_col] = "90/60"  # Critical but viable
                                        bp_corrections['critical_viable'] += 1
                                    corrected = True
                                else:
                                    # Values are in reasonable range, just ensure systolic > diastolic
                                    if systolic <= diastolic:
                                        diastolic = max(20, systolic - 10)
                                        cleaned_vitals.loc[idx, bp_col] = f"{int(systolic)}/{int(diastolic)}"
                                        bp_corrections['data_errors'] += 1
                                        corrected = True

                        except (ValueError, IndexError):
                            corrected = False

                    if not corrected:
                        # Invalid format - extract numbers and create valid format
                        import re
                        numbers = re.findall(r'\d+', bp_str)
                        if len(numbers) >= 2:
                            systolic = int(numbers[0])
                            diastolic = int(numbers[1])
                        elif len(numbers) == 1:
                            systolic = int(numbers[0])
                            diastolic = max(20, int(systolic * 0.67))
                        else:
                            # No numbers found, use baseline based on patient status
                            patient_status = is_patient_viable(row, exclude_column='pressure')
                            if patient_status == 'normal':
                                systolic, diastolic = 120, 80
                            elif patient_status == 'dead':
                                systolic, diastolic = 60, 40
                            else:
                                systolic, diastolic = 90, 60

                        # Ensure reasonable values
                        systolic = max(30, min(systolic, 300))
                        diastolic = max(20, min(diastolic, 200))
                        if systolic <= diastolic:
                            diastolic = max(20, systolic - 10)

                        cleaned_vitals.loc[idx, bp_col] = f"{systolic}/{diastolic}"
                        bp_corrections['formats_fixed'] += 1

        cleaning_stats['corrected_values']['pressure'] = bp_corrections
        cleaning_stats['invalid_formats_fixed']['pressure'] = bp_corrections['formats_fixed']

        if verbose and sum(bp_corrections.values()) > 0:
            print_heading_3(f"BLOOD PRESSURE CORRECTION:")
            print(f"   • Normal range target: 90-140/60-90 mmHg (baseline: 120/80)")
            print(f"   • Critical viable range: 70-89/40-59 mmHg (correction: 90/60)")
            print(f"   • Data errors corrected: {bp_corrections['data_errors']:,}")
            print(f"   • Critical viable corrections: {bp_corrections['critical_viable']:,}")
            print(f"   • Invalid formats fixed: {bp_corrections['formats_fixed']:,}")

    # =============================================================================
    # FINAL STATISTICS
    # =============================================================================
    # Calculate totals
    total_data_errors = sum(
        stats.get('data_errors', 0) for stats in cleaning_stats['corrected_values'].values()
    )
    total_critical_corrections = sum(
        stats.get('critical_viable', 0) for stats in cleaning_stats['corrected_values'].values()
    )
    total_death_preserved = sum(
        stats.get('death_preserved', 0) for stats in cleaning_stats['corrected_values'].values()
    )
    total_formats_fixed = sum(cleaning_stats['invalid_formats_fixed'].values())

    cleaning_stats['data_errors_corrected'] = total_data_errors
    cleaning_stats['critical_but_viable_corrections'] = total_critical_corrections
    cleaning_stats['death_cases_preserved'] = total_death_preserved

    if verbose:
        print_heading_3(f"INTELLIGENT CORRECTION SUMMARY:")
        print(f"   • Original records: {cleaning_stats['original_records']:,}")
        print(f"   • Records preserved: {len(cleaned_vitals):,} (100%)")
        print(f"   • Data entry errors corrected to normal: {total_data_errors:,}")
        print(f"   • Critical but viable corrections: {total_critical_corrections:,}")
        print(f"   • Death cases preserved: {total_death_preserved:,}")
        print(f"   • Invalid formats fixed: {total_formats_fixed:,}")
        print(f"   • ALL DATA PRESERVED with medically intelligent corrections")

        # Individual column statistics
        print_heading_3(f"CORRECTIONS BY VITAL SIGN:")
        for vital, stats in cleaning_stats['corrected_values'].items():
            total_corrections = sum(stats.values()) if isinstance(stats, dict) else stats
            if total_corrections > 0:
                print(f"   • {vital.upper()}:")
                if isinstance(stats, dict):
                    for correction_type, count in stats.items():
                        if count > 0:
                            type_desc = {
                                'data_errors': 'data errors → normal',
                                'critical_viable': 'critical but viable',
                                'death_preserved': 'death cases preserved',
                                'formats_fixed': 'formats fixed'
                            }.get(correction_type, correction_type)
                            print(f"     - {count:,} {type_desc}")

    if verbose:
        print_heading_3("✓ Intelligent vitals correction completed successfully")
        print(f"✓ ALL records preserved with context-aware medical corrections")
        print(f"✓ Dataset ready for analysis with clinically plausible vital signs")

    return cleaned_vitals

vitals = clean_vitals_data(vitals)


                                      Data Loaded Successfully                                      

      Vitals Data Cleaning - Intelligent Medical Correction 

Original dataset: 744 vital sign records
----------------------------------------

OXYGEN SATURATION CORRECTION:
-----------------------------
   • Normal range target: 95-100% (baseline: 98%)
   • Critical viable range: 70-94% (correction: 88%)
   • Data errors corrected to normal: 0
   • Critical but viable corrections: 0
   • Death cases preserved: 6

BLOOD GLUCOSE CORRECTION:
-------------------------
   • Normal range target: 70-180 mg/dL (baseline: 100 mg/dL)
   • Critical viable range: 40-69 mg/dL (correction: 60 mg/dL)
   • Data errors corrected to normal: 136
   • Critical but viable corrections: 2
   • Death cases preserved: 1

TEMPERATURE CORRECTION:
-----------------------
   • Normal range target: 36-38°C (baseline: 37°C)
   • Critical viable range: 34-36°C (correction: 35°C)
   • Data errors corrected to norma

## Dashboard

In [ ]:
# COMPLETE GEO-HEALTH DASHBOARD - FIXED VERSION
# No truncation issues, complete implementation

# Installation and setup
print("Installing required packages...")
!pip install streamlit plotly pyngrok folium streamlit-folium scikit-learn seaborn matplotlib

# Create the complete geo-health dashboard
dashboard_content = '''
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime, timedelta
import warnings
import os
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import folium
from streamlit_folium import st_folium
warnings.filterwarnings('ignore')

# Configure page
st.set_page_config(
    page_title="Regional Health Analytics Dashboard",
    layout="wide"
)


st.markdown("""
<style>
.main-header {
    font-size: 2rem !important;
    font-weight: bold;
    text-align: center;
    color: #2c3e50;
    margin-bottom: 2rem;
}
.section-header {
    font-size: 1.5rem !important;
    font-weight: bold;
    color: #34495e;
    margin: 1.5rem 0 1rem 0 !important;
    border-bottom: 2px solid #3498db;
    padding-bottom: 0.5rem;
}
.metric-card {
    background: #f8f9fa;
    padding: 1.5rem;
    border-radius: 10px;
    border-left: 4px solid #3498db;
    margin: 1rem 0;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.insight-box {
    background: #e8f4fd;
    padding: 1.5rem;
    border-radius: 10px;
    border-left: 4px solid #2980b9;
    margin: 1rem 0;
}
.pattern-card {
    background: #fff;
    padding: 1rem;
    border-radius: 8px;
    border-left: 4px solid #e74c3c;
    margin: 0.5rem 0;
    box-shadow: 0 1px 3px rgba(0,0,0,0.1);
}
.hospital-analysis {
    background: #f9f9f9;
    padding: 1.5rem;
    border-radius: 10px;
    margin: 1rem 0;
    border: 1px solid #ddd;
}
.subHeader {
    color: #2c3e50;
    font-size: 1.3rem;
    text-decoration: underline;
}
</style>
""", unsafe_allow_html=True)

@st.cache_data
def load_data():
    """Load medical data with error handling"""
    base_paths = [
        "/content/drive/MyDrive/Xpertnurse Project/synthetic_medical_data/",
        "/content/drive/My Drive/Xpertnurse Project/synthetic_medical_data/",
        "/content/synthetic_medical_data/",
        "./"
    ]

    data_path = None
    for path in base_paths:
        if os.path.exists(path + "patients.csv"):
            data_path = path
            break

    if data_path is None:
        return None, "Data files not found"

    try:
        # Load all available data
        data = {}
        files = {
            'patients': 'patients.csv',
            'vitals': 'vitals.csv',
            'hospitals': 'hospitals.csv',
            'doctors': 'doctors.csv',
            'appointments': 'appointments.csv',
            'surgeries': 'surgeries.csv',
            'drug_records': 'drug_records.csv',
            'drugs': 'drugs.csv',
            'hospital_patient': 'hospital_patient.csv',
            'medical_tests': 'medical_tests.csv',
            'doctor_hospital': 'doctor_hospital.csv',
            'drug_patient': 'drug_patient.csv'
        }

        for key, filename in files.items():
            try:
                df = pd.read_csv(data_path + filename)
                data[key] = df
            except:
                data[key] = pd.DataFrame()

        # Convert date columns
        date_columns = {
            'vitals': ['created_at'],
            'appointments': ['appointment_datetime'],
            'surgeries': ['created_at'],
            'medical_tests': ['created_at'],
            'hospital_patient': ['created_at']
        }

        for table, cols in date_columns.items():
            if not data[table].empty:
                for col in cols:
                    if col in data[table].columns:
                        data[table][col] = pd.to_datetime(data[table][col], errors='coerce')

        return data, None

    except Exception as e:
        return None, f"Error loading data: {str(e)}"

def create_geographic_clusters(data):
    """Group patients by geographic proximity using hospital locations"""

    if data['hospitals'].empty:
        return pd.DataFrame()

    # Get hospital coordinates
    hospital_coords = data['hospitals'][['id', 'name', 'latitude', 'longitude']].copy()
    hospital_coords = hospital_coords.dropna(subset=['latitude', 'longitude'])

    if len(hospital_coords) < 2:
        return hospital_coords.assign(cluster=0)

    # Perform clustering on hospital locations
    coords = hospital_coords[['latitude', 'longitude']].values
    scaler = StandardScaler()
    coords_scaled = scaler.fit_transform(coords)

    # Use DBSCAN for clustering
    clustering = DBSCAN(eps=0.1, min_samples=1).fit(coords_scaled)
    hospital_coords['cluster'] = clustering.labels_

    return hospital_coords

def analyze_regional_patterns(data):
    """Analyze health patterns by geographic regions"""

    clusters = create_geographic_clusters(data)
    if clusters.empty:
        return {}

    regional_analysis = {}

    for cluster_id in clusters['cluster'].unique():
        cluster_hospitals = clusters[clusters['cluster'] == cluster_id]
        hospital_ids = cluster_hospitals['id'].tolist()

        # Get patients associated with these hospitals
        region_patients = []
        if not data['hospital_patient'].empty:
            region_patients = data['hospital_patient'][
                data['hospital_patient']['hospital_id'].isin(hospital_ids)
            ]['patient_id'].unique()

        if len(region_patients) == 0:
            continue

        # Analyze vitals for this region
        region_vitals = data['vitals'][data['vitals']['patient_id'].isin(region_patients)]

        # Analyze surgeries
        region_surgeries = pd.DataFrame()
        if not data['surgeries'].empty and not data['hospital_patient'].empty:
            hp_records = data['hospital_patient'][
                data['hospital_patient']['hospital_id'].isin(hospital_ids)
            ]['id'].tolist()
            region_surgeries = data['surgeries'][
                data['surgeries']['hospital_patient_id'].isin(hp_records)
            ]

        # Analyze drug usage
        region_drugs = data['drug_records'][
            data['drug_records']['patient_id'].isin(region_patients)
        ] if not data['drug_records'].empty else pd.DataFrame()

        regional_analysis[f'Region_{cluster_id}'] = {
            'hospitals': cluster_hospitals,
            'patient_ids': region_patients,
            'vitals': region_vitals,
            'surgeries': region_surgeries,
            'drug_records': region_drugs,
            'patient_count': len(region_patients),
            'hospital_count': len(cluster_hospitals)
        }

    return regional_analysis

def generate_insights_text(regional_analysis, data):
    """Generate comprehensive text-based insights"""

    insights = []

    # Overall regional summary
    total_regions = len(regional_analysis)
    total_patients = sum([region['patient_count'] for region in regional_analysis.values()])

    insights.append(f"""
    <h3 style="color: #2c3e50; font-size: 1.4rem; font-weight: bold; margin-top: 1.5rem; margin-bottom: 1rem;">Regional Health Distribution Analysis</h3>
    <p>The healthcare system analysis reveals {total_regions} distinct geographic regions serving {total_patients:,} total patients.
     Hospitals that are geographically close to each other (based on their latitude/longitude coordinates) are grouped into the same region.</p>
    """)

    # Regional breakdown
    for region_name, region_data in regional_analysis.items():
        region_num = region_name.split('_')[1]

        # Hospital analysis for this region
        hospital_names = ', '.join(region_data['hospitals']['name'].tolist())

        # Vitals analysis
        if not region_data['vitals'].empty:
            avg_temp = region_data['vitals']['temperature'].mean()
            avg_hr = region_data['vitals']['heartbeat'].mean()
            avg_oxygen = region_data['vitals']['oxygen'].mean()
            avg_glucose = region_data['vitals']['glucose'].mean()

            # Health indicators
            temp_abnormal = len(region_data['vitals'][
                (region_data['vitals']['temperature'] < 35.5) |
                (region_data['vitals']['temperature'] > 38.0)
            ])
            hr_abnormal = len(region_data['vitals'][
                (region_data['vitals']['heartbeat'] < 60) |
                (region_data['vitals']['heartbeat'] > 100)
            ])

        # Surgery analysis
        if not region_data['surgeries'].empty:
            surgery_types = region_data['surgeries']['procedure_name'].value_counts()
            most_common_surgery = surgery_types.index[0] if len(surgery_types) > 0 else "None"

        # Drug analysis
        if not region_data['drug_records'].empty and not data['drugs'].empty:
            drug_usage = region_data['drug_records'].merge(
                data['drugs'][['id', 'name']], left_on='drug_id', right_on='id'
            )
            if not drug_usage.empty:
                common_drugs = drug_usage['name'].value_counts().head(3)
                drug_list = ', '.join(common_drugs.index.tolist())

        # Build the HTML content step by step to avoid f-string issues
        html_content = f"""
        <h4 style="color: #34495e; font-size: 1.5rem; font-weight: bold; margin-top: 1.0rem; margin-bottom: 0.8rem;">Region {region_num} Analysis</h4>
        <p style="margin-bottom: 0.5rem;"><strong class="subHeader">Healthcare Facilities:</strong> {hospital_names}</p>
        <p style="margin-bottom: 0.8rem;"><strong class="subHeader">Patient Population:</strong> {region_data['patient_count']} patients across {region_data['hospital_count']} facilities</p>
        """

        # Add vitals section if data exists
        if not region_data['vitals'].empty:
            html_content += f"""
        <p style="margin-top: 1rem; margin-bottom: 0.5rem;"><strong class="subHeader">Vital Signs Overview:</strong></p>
        <ul>
            <li>Average temperature: {avg_temp:.1f}°C</li>
            <li>Average heart rate: {avg_hr:.0f} BPM</li>
            <li>Average oxygen saturation: {avg_oxygen:.1f}%</li>
            <li>Average glucose levels: {avg_glucose:.0f} mg/dL</li>
            <li>Abnormal temperature readings: {temp_abnormal}</li>
            <li>Irregular heart rate measurements: {hr_abnormal}</li>
        </ul>
        """

        # Add surgery section if data exists
        if not region_data['surgeries'].empty:
            html_content += f"""
        <p style="margin-top: 1rem; margin-bottom: 0.5rem;"><strong class="subHeader">Surgical Activity:</strong></p>
        <ul>
            <li>Total procedures performed: {len(region_data['surgeries'])}</li>
            <li>Most frequent intervention: {most_common_surgery}</li>
        </ul>
        """

        # Add medication section if data exists
        if not region_data['drug_records'].empty and not data['drugs'].empty:
            html_content += f"""
        <p style="margin-top: 1rem; margin-bottom: 0.5rem;"><strong class="subHeader">Medication Patterns:</strong></p>
        <ul>
            <li>Total prescriptions: {len(region_data['drug_records'])}</li>
            <li>Most commonly prescribed medications: {drug_list}</li>
        </ul>
        """

        insights.append(html_content)

    return insights

def analyze_hospital_details(data):
    """Detailed analysis of individual hospitals"""

    hospital_analyses = {}

    for _, hospital in data['hospitals'].iterrows():
        # Get patients for this hospital
        hospital_patients = []
        if not data['hospital_patient'].empty:
            hospital_patients = data['hospital_patient'][
                data['hospital_patient']['hospital_id'] == hospital['id']
            ]['patient_id'].unique()

        # Get doctors at this hospital
        hospital_doctors = []
        if not data['doctor_hospital'].empty:
            doctor_ids = data['doctor_hospital'][
                data['doctor_hospital']['hospital_id'] == hospital['id']
            ]['doctor_id'].unique()
            hospital_doctors = data['doctors'][
                data['doctors']['id'].isin(doctor_ids)
            ] if not data['doctors'].empty else pd.DataFrame()

        # Analyze specialties
        specialties = []
        if not hospital_doctors.empty and 'specialty' in hospital_doctors.columns:
            specialties = hospital_doctors['specialty'].value_counts()

        # Analyze surgeries at this hospital
        hospital_surgeries = pd.DataFrame()
        if not data['surgeries'].empty and not data['hospital_patient'].empty:
            hp_records = data['hospital_patient'][
                data['hospital_patient']['hospital_id'] == hospital['id']
            ]['id'].tolist()
            hospital_surgeries = data['surgeries'][
                data['surgeries']['hospital_patient_id'].isin(hp_records)
            ]

        # Analyze patient vitals
        hospital_vitals = pd.DataFrame()
        if len(hospital_patients) > 0 and not data['vitals'].empty:
            hospital_vitals = data['vitals'][
                data['vitals']['patient_id'].isin(hospital_patients)
            ]

        hospital_analyses[hospital['name']] = {
            'basic_info': hospital,
            'patient_count': len(hospital_patients),
            'doctor_count': len(hospital_doctors) if not hospital_doctors.empty else 0,
            'specialties': specialties,
            'surgeries': hospital_surgeries,
            'vitals': hospital_vitals,
            'surgery_types': hospital_surgeries['procedure_name'].value_counts() if not hospital_surgeries.empty else pd.Series()
        }

    return hospital_analyses

def identify_surgical_patterns(data):
    """Identify surgical procedure patterns across regions"""

    if data['surgeries'].empty:
        return {}

    patterns = {}

    # Overall surgical patterns
    surgery_counts = data['surgeries']['procedure_name'].value_counts()

    patterns['overall'] = {
        'total_surgeries': len(data['surgeries']),
        'procedure_types': len(surgery_counts),
        'most_common': surgery_counts.head(5),
        'analysis': f"""
        Analysis of {len(data['surgeries'])} surgical procedures reveals {len(surgery_counts)} distinct procedure types.
        The most frequently performed surgery is '{surgery_counts.index[0]}' with {surgery_counts.iloc[0]} cases,
        suggesting this is a primary health concern requiring surgical intervention.
        """
    }

    # Temporal patterns
    if 'created_at' in data['surgeries'].columns:
        surgery_temporal = data['surgeries'].copy()
        surgery_temporal['month'] = surgery_temporal['created_at'].dt.to_period('M')
        monthly_surgeries = surgery_temporal.groupby('month').size()

        patterns['temporal'] = {
            'monthly_average': monthly_surgeries.mean(),
            'peak_month': monthly_surgeries.idxmax(),
            'peak_count': monthly_surgeries.max(),
            'trend_analysis': f"""
            Surgical activity shows an average of {monthly_surgeries.mean():.1f} procedures per month.
            The highest surgical volume occurred in {monthly_surgeries.idxmax()} with {monthly_surgeries.max()} procedures.
            """
        }

    # Hospital-specific surgical patterns
    if not data['hospital_patient'].empty and not data['hospitals'].empty:
        try:
            surgery_hospital = data['surgeries'].merge(
                data['hospital_patient'],
                left_on='hospital_patient_id',
                right_on='id'
            ).merge(
                data['hospitals'][['id', 'name']],
                left_on='hospital_id',
                right_on='id'
            )

            hospital_surgery_counts = surgery_hospital.groupby('name')['procedure_name'].count().sort_values(ascending=False)

            patterns['by_hospital'] = {
                'hospital_rankings': hospital_surgery_counts,
                'analysis': f"""
                Surgical volume distribution shows {hospital_surgery_counts.iloc[0]} procedures at the leading facility
                ({hospital_surgery_counts.index[0]}), indicating this hospital serves as a major surgical center.
                """
            }
        except:
            patterns['by_hospital'] = {'analysis': 'Hospital-specific surgical analysis unavailable due to data connectivity issues.'}

    return patterns

def analyze_medication_patterns(data):
    """Analyze medication patterns - combines all medication data"""

    patterns = {}

    # Combine drug_records and drug_patient into one dataset
    all_medications = pd.DataFrame()

    if not data['drug_records'].empty and not data['drugs'].empty:
        drug_records_with_names = data['drug_records'].merge(
            data['drugs'][['id', 'name']],
            left_on='drug_id',
            right_on='id'
        )
        all_medications = pd.concat([all_medications, drug_records_with_names[['patient_id', 'drug_id', 'name']]])

    if not data['drug_patient'].empty and not data['drugs'].empty:
        drug_patient_with_names = data['drug_patient'].merge(
            data['drugs'][['id', 'name']],
            left_on='drug_id',
            right_on='id'
        )
        all_medications = pd.concat([all_medications, drug_patient_with_names[['patient_id', 'drug_id', 'name']]])

    # Remove duplicates (same patient + same drug)
    all_medications = all_medications.drop_duplicates(subset=['patient_id', 'drug_id'])

    if not all_medications.empty:
        medication_counts = all_medications['name'].value_counts()

        patterns['overall'] = {
            'total_medications': len(all_medications),
            'unique_medications': len(medication_counts),
            'most_prescribed': medication_counts.head(5),
            'analysis': f"""
            Medication analysis reveals {len(all_medications)} total medication records across {len(medication_counts)} different medications.
            The most frequently prescribed medication is '{medication_counts.index[0]}' with {medication_counts.iloc[0]} patients.
            """
        }

        # Hospital-specific patterns
        hospital_med_patterns = {}
        if not data['hospital_patient'].empty:
            for _, hospital in data['hospitals'].iterrows():
                hospital_patients = data['hospital_patient'][
                    data['hospital_patient']['hospital_id'] == hospital['id']
                ]['patient_id'].unique()

                hospital_meds = all_medications[all_medications['patient_id'].isin(hospital_patients)]
                if not hospital_meds.empty:
                    hospital_med_patterns[hospital['name']] = hospital_meds['name'].value_counts().head(5)

        patterns['hospital_patterns'] = hospital_med_patterns

        # Patient medication burden
        patient_med_counts = all_medications.groupby('patient_id').size()

        patterns['patient_burden'] = {
            'average_meds_per_patient': patient_med_counts.mean(),
            'max_meds_single_patient': patient_med_counts.max(),
            'patients_multiple_meds': len(patient_med_counts[patient_med_counts > 1]),
            'analysis': f"""
            Patient medication burden analysis shows an average of {patient_med_counts.mean():.1f} medications per patient.
            {len(patient_med_counts[patient_med_counts > 1])} patients are on multiple medications, with one patient receiving
            up to {patient_med_counts.max()} different medications, indicating complex health conditions requiring
            multi-drug therapeutic approaches.
            """
        }

    return patterns

def main():
    st.markdown('<p class="main-header">Regional Healthcare Analytics Dashboard</p>', unsafe_allow_html=True)

    # Load data
    data, error = load_data()

    if error:
        st.error(f"Data loading failed: {error}")
        st.stop()

    # Sidebar filters
    st.sidebar.header("Analysis Configuration")

    # Date range filter
    if not data['vitals'].empty and 'created_at' in data['vitals'].columns:
        min_date = data['vitals']['created_at'].min().date()
        max_date = data['vitals']['created_at'].max().date()

        date_range = st.sidebar.date_input(
            "Analysis Period",
            value=[min_date, max_date],
            min_value=min_date,
            max_value=max_date
        )

        if len(date_range) == 2:
            start_date, end_date = date_range
            # Apply filters to relevant tables
            data['vitals'] = data['vitals'][
                (data['vitals']['created_at'].dt.date >= start_date) &
                (data['vitals']['created_at'].dt.date <= end_date)
            ]

    # Analysis type selection
    analysis_types = st.sidebar.multiselect(
        "Analysis Components",
        ["Regional Patterns", "Hospital Analysis", "Surgical Patterns", "Medication Patterns"],
        default=["Regional Patterns", "Hospital Analysis", "Surgical Patterns", "Medication Patterns"]
    )

    # Main analysis
    with st.spinner("Generating comprehensive health analytics..."):

        # Initialize regional_analysis outside conditional blocks
        regional_analysis = {}

        # Regional analysis
        if "Regional Patterns" in analysis_types:
            st.markdown('<p class="section-header">Regional Health Pattern Analysis</p>', unsafe_allow_html=True)

            regional_analysis = analyze_regional_patterns(data)

            # Add region filter dropdown
            if regional_analysis:
                all_regions = list(regional_analysis.keys())
                region_options = ['All Regions'] + all_regions
                selected_regions = st.multiselect(
                    "Select Regions to Display",
                    options=region_options,
                    default=['All Regions']
                )

                # Determine which regions to show
                if 'All Regions' in selected_regions:
                    regions_to_show = all_regions
                else:
                    regions_to_show = selected_regions

                # Filter regional analysis based on selection
                filtered_regional_analysis = {k: v for k, v in regional_analysis.items() if k in regions_to_show}
                insights = generate_insights_text(filtered_regional_analysis, data)

                for insight in insights:
                    st.markdown(insight, unsafe_allow_html=True)
            else:
                insights = generate_insights_text(regional_analysis, data)
                for insight in insights:
                    st.markdown(insight, unsafe_allow_html=True)

            # Regional comparison charts
            if regional_analysis:
                region_summary = []
                for region_name, region_data in regional_analysis.items():
                    vitals_count = len(region_data['vitals'])
                    surgery_count = len(region_data['surgeries'])
                    drug_count = len(region_data['drug_records'])

                    region_summary.append({
                        'Region': region_name.replace('_', ' '),
                        'Patients': region_data['patient_count'],
                        'Hospitals': region_data['hospital_count'],
                        'Vitals Readings': vitals_count,
                        'Surgeries': surgery_count,
                        'Drug Prescriptions': drug_count
                    })

                summary_df = pd.DataFrame(region_summary)

                if not summary_df.empty:
                    # Create comparison charts
                    col1, col2 = st.columns(2)

                    with col1:
                        fig_patients = px.bar(
                            summary_df,
                            x='Region',
                            y='Patients',
                            title='Patient Distribution by Region',
                            color='Patients',
                            color_continuous_scale='Tealgrn'
                        )
                        st.plotly_chart(fig_patients, use_container_width=True)

                    with col2:
                        fig_activity = px.bar(
                            summary_df.melt(id_vars=['Region'],
                                           value_vars=['Vitals Readings', 'Surgeries', 'Drug Prescriptions'],
                                           var_name='Activity Type',
                                           value_name='Count'),
                            x='Region',
                            y='Count',
                            color='Activity Type',
                            title='Healthcare Activity by Region',
                            barmode='group'
                        )
                        st.plotly_chart(fig_activity, use_container_width=True)

        # Hospital analysis
        if "Hospital Analysis" in analysis_types:
            st.markdown('<p class="section-header">Individual Hospital Analysis</p>', unsafe_allow_html=True)

            hospital_analyses = analyze_hospital_details(data)

            for hospital_name, analysis in hospital_analyses.items():
                with st.expander(f"{hospital_name} - Detailed Analysis"):

                    col1, col2, col3 = st.columns(3)

                    with col1:
                        st.markdown(f"""
                        <div class="metric-card">
                            <h4>Patient Population</h4>
                            <h3>{analysis['patient_count']:,}</h3>
                            <p>Total patients served</p>
                        </div>
                        """, unsafe_allow_html=True)

                    with col2:
                        st.markdown(f"""
                        <div class="metric-card">
                            <h4>Medical Staff</h4>
                            <h3>{analysis['doctor_count']}</h3>
                            <p>Doctors affiliated</p>
                        </div>
                        """, unsafe_allow_html=True)

                    with col3:
                        surgery_count = len(analysis['surgeries'])
                        st.markdown(f"""
                        <div class="metric-card">
                            <h4>Surgical Procedures</h4>
                            <h3>{surgery_count}</h3>
                            <p>Total procedures performed</p>
                        </div>
                        """, unsafe_allow_html=True)

                    # Hospital insights
                    location_info = f"Located at coordinates {analysis['basic_info']['latitude']:.4f}, {analysis['basic_info']['longitude']:.4f}"

                    specialties_text = "No specialty information available"
                    if not analysis['specialties'].empty:
                        top_specialties = analysis['specialties'].head(3)
                        specialties_text = f"Primary medical specialties include {', '.join(top_specialties.index)}"

                    vitals_analysis = "No vitals monitoring data available"
                    if not analysis['vitals'].empty:
                        avg_readings = len(analysis['vitals']) / analysis['patient_count'] if analysis['patient_count'] > 0 else 0
                        vitals_analysis = f"Monitoring {len(analysis['vitals'])} vital sign readings across the patient population (average {avg_readings:.1f} readings per patient)"

                    surgery_analysis = "No surgical procedures recorded"
                    if not analysis['surgeries'].empty:
                        most_common_surgery = analysis['surgery_types'].index[0] if len(analysis['surgery_types']) > 0 else "Unknown"
                        surgery_analysis = f"Performed {len(analysis['surgeries'])} surgical procedures, with '{most_common_surgery}' being the most common intervention"

                    hospital_summary = f"""
                    <div style="background: #f5f5f5; padding: 1rem; border-radius: 8px; margin: 1rem 0;">
                    <h4 style="color: #2c3e50; margin-bottom: 0.5rem;">{hospital_name} Operational Summary</h4>
                    <p>Located at coordinates {analysis['basic_info']['latitude']:.4f}, {analysis['basic_info']['longitude']:.4f}. This facility serves {analysis['patient_count']} patients with a medical staff of {analysis['doctor_count']} doctors.
                    {specialties_text}.</p>

                    <p><strong>Patient Care Metrics:</strong> {vitals_analysis}.</p>

                    <p><strong>Surgical Services:</strong> {surgery_analysis}.</p>
                    </div>
                    """

                    st.markdown(hospital_summary, unsafe_allow_html=True)

                    # Charts for this hospital
                    col_chart1, col_chart2 = st.columns(2)

                    with col_chart1:
                        if not analysis['vitals'].empty:
                            vitals_summary = {
                                'Temperature': analysis['vitals']['temperature'].mean(),
                                'Heart Rate': analysis['vitals']['heartbeat'].mean(),
                                'Oxygen': analysis['vitals']['oxygen'].mean(),
                                'Glucose': analysis['vitals']['glucose'].mean()
                            }

                            fig_vitals = px.bar(
                                x=list(vitals_summary.keys()),
                                y=list(vitals_summary.values()),
                                title=f"{hospital_name} - Average Vital Signs",
                                labels={'x': 'Vital Sign', 'y': 'Average Value'}
                            )
                            st.plotly_chart(fig_vitals, use_container_width=True)

                    with col_chart2:
                        # Hospital location map
                        if pd.notna(analysis['basic_info']['latitude']) and pd.notna(analysis['basic_info']['longitude']):
                            lat = float(analysis['basic_info']['latitude'])
                            lon = float(analysis['basic_info']['longitude'])

                            hospital_map = folium.Map(location=[lat, lon], zoom_start=15)
                            folium.Marker(
                                [lat, lon],
                                popup=f"{hospital_name}<br>Patients: {analysis['patient_count']}<br>Doctors: {analysis['doctor_count']}",
                                icon=folium.Icon(color='blue', icon='hospital-o', prefix='fa')
                            ).add_to(hospital_map)

                            st_folium(hospital_map, width=400, height=300)

        # Surgical patterns
        if "Surgical Patterns" in analysis_types:
            st.markdown('<p class="section-header">Pattern Details</p>', unsafe_allow_html=True)
            st.markdown('<p style="font-size: 1.2rem; font-weight: bold; color: #e74c3c;">Surgical Procedure Patterns:</p>', unsafe_allow_html=True)

            # Add surgery search feature
            if not data['surgeries'].empty:
                all_surgeries = sorted(data['surgeries']['procedure_name'].unique().tolist())

                selected_surgery = st.selectbox(
                    "Search for a specific surgery procedure:",
                    options=[''] + all_surgeries,
                    format_func=lambda x: "Select a surgery..." if x == '' else x
                )

                if selected_surgery and selected_surgery != '':
                    st.markdown(f'<h4 style="color: #34495e; margin-top: 1rem;">Results for: {selected_surgery}</h4>', unsafe_allow_html=True)

                    # Get hospital-specific data for this surgery
                    if not data['hospital_patient'].empty and not data['hospitals'].empty:
                        surgery_hospital_data = data['surgeries'][
                            data['surgeries']['procedure_name'] == selected_surgery
                        ].merge(
                            data['hospital_patient'][['id', 'hospital_id']],
                            left_on='hospital_patient_id',
                            right_on='id'
                        ).merge(
                            data['hospitals'][['id', 'name']],
                            left_on='hospital_id',
                            right_on='id'
                        )

                        if not surgery_hospital_data.empty:
                            hospital_counts = surgery_hospital_data.groupby('name').size().reset_index(name='Count')

                            fig_surgery_search = px.bar(
                                hospital_counts,
                                x='name',
                                y='Count',
                                title=f'Number of {selected_surgery} Procedures by Hospital',
                                labels={'name': 'Hospital', 'Count': 'Number of Procedures'},
                                color='Count',
                                color_continuous_scale='Reds'
                            )
                            fig_surgery_search.update_xaxes(tickangle=45)
                            st.plotly_chart(fig_surgery_search, use_container_width=True)
                        else:
                            st.info("No hospital data available for this surgery.")
                    else:
                        st.info("Hospital data not available.")

            surgical_patterns = identify_surgical_patterns(data)

            if surgical_patterns:
                # Add surgical volume by region FIRST
                if regional_analysis:
                    region_surgery_data = []
                    for region_name, region_data in regional_analysis.items():
                        region_surgery_data.append({
                            'Region': region_name.replace('_', ' '),
                            'Surgical Volume': len(region_data['surgeries'])
                        })

                    if region_surgery_data:
                        surgery_region_df = pd.DataFrame(region_surgery_data)
                        fig_regional_surgeries = px.bar(
                            surgery_region_df,
                            x='Region',
                            y='Surgical Volume',
                            title='Surgical Volume by Region',
                            color='Surgical Volume',
                            color_continuous_scale='Tealgrn'
                        )
                        st.plotly_chart(fig_regional_surgeries, use_container_width=True)

                # Overall patterns
                if 'overall' in surgical_patterns:
                    st.markdown(f'<div class="insight-box">{surgical_patterns["overall"]["analysis"]}</div>', unsafe_allow_html=True)

                    if not surgical_patterns['overall']['most_common'].empty:
                        fig_surgeries = px.bar(
                            x=surgical_patterns['overall']['most_common'].values,
                            y=surgical_patterns['overall']['most_common'].index,
                            orientation='h',
                            title='Most Common Surgical Procedures',
                            labels={'x': 'Number of Procedures', 'y': 'Procedure Type'}
                        )
                        st.plotly_chart(fig_surgeries, use_container_width=True)

                # Temporal patterns
                if 'temporal' in surgical_patterns:
                    st.markdown(f'<div class="insight-box">{surgical_patterns["temporal"]["trend_analysis"]}</div>', unsafe_allow_html=True)

                # Hospital patterns
                if 'by_hospital' in surgical_patterns and 'hospital_rankings' in surgical_patterns['by_hospital']:
                    st.markdown(f'<div class="insight-box">Surgical volume distribution shows {surgical_patterns["by_hospital"]["hospital_rankings"].iloc[0]} procedures at the leading facility ({surgical_patterns["by_hospital"]["hospital_rankings"].index[0]}).</div>', unsafe_allow_html=True)

                    fig_hospital_surgeries = px.bar(
                        x=surgical_patterns['by_hospital']['hospital_rankings'].values,
                        y=surgical_patterns['by_hospital']['hospital_rankings'].index,
                        orientation='h',
                        title='Surgical Volume by Hospital',
                        labels={'x': 'Number of Procedures', 'y': 'Hospital'}
                    )
                    st.plotly_chart(fig_hospital_surgeries, use_container_width=True)

        # Medication patterns
        if "Medication Patterns" in analysis_types:
            st.markdown('<p style="font-size: 1.2rem; font-weight: bold; color: #e74c3c;">Medication Prescription Patterns:</p>', unsafe_allow_html=True)

            medication_patterns = analyze_medication_patterns(data)

            if medication_patterns:
                # Most prescribed medications by region FIRST
                if regional_analysis and not data['drugs'].empty:
                    # Combine all medication data
                    all_meds = pd.DataFrame()

                    if not data['drug_records'].empty:
                        dr = data['drug_records'].merge(
                            data['drugs'][['id', 'name']],
                            left_on='drug_id',
                            right_on='id'
                        )
                        all_meds = pd.concat([all_meds, dr[['patient_id', 'drug_id', 'name']]])

                    if not data['drug_patient'].empty:
                        dp = data['drug_patient'].merge(
                            data['drugs'][['id', 'name']],
                            left_on='drug_id',
                            right_on='id'
                        )
                        all_meds = pd.concat([all_meds, dp[['patient_id', 'drug_id', 'name']]])

                    all_meds = all_meds.drop_duplicates(subset=['patient_id', 'drug_id'])

                    if not all_meds.empty:
                        region_prescription_data = []

                        for region_name, region_data in regional_analysis.items():
                            region_meds = all_meds[
                                all_meds['patient_id'].isin(region_data['patient_ids'])
                            ]

                            if not region_meds.empty:
                                top_meds = region_meds['name'].value_counts().head(5)
                                for med, count in top_meds.items():
                                    region_prescription_data.append({
                                        'Region': region_name.replace('_', ' '),
                                        'Medication': med[:30],
                                        'Prescriptions': count
                                    })

                        if region_prescription_data:
                            region_prescription_df = pd.DataFrame(region_prescription_data)
                            fig_regional_prescriptions = px.bar(
                                region_prescription_df,
                                x='Region',
                                y='Prescriptions',
                                color='Medication',
                                title='Top 5 Medications by Region',
                                barmode='group'
                            )
                            fig_regional_prescriptions.update_xaxes(tickangle=45)
                            st.plotly_chart(fig_regional_prescriptions, use_container_width=True)

                # Overall medications patterns
                if 'overall' in medication_patterns:
                    st.markdown(f'<div class="insight-box">{medication_patterns["overall"]["analysis"]}</div>', unsafe_allow_html=True)

                    if 'most_prescribed' in medication_patterns['overall']:
                        fig_meds = px.bar(
                            x=medication_patterns['overall']['most_prescribed'].values,
                            y=medication_patterns['overall']['most_prescribed'].index,
                            orientation='h',
                            title='Top 5 Most Prescribed Medications Overall',
                            labels={'x': 'Number of Patients', 'y': 'Medication'},
                            color=medication_patterns['overall']['most_prescribed'].values,
                            color_continuous_scale='Tealgrn'
                        )
                        st.plotly_chart(fig_meds, use_container_width=True)

                # Hospital medication patterns - Top 5 per hospital
                if 'hospital_patterns' in medication_patterns and medication_patterns['hospital_patterns']:
                    st.markdown('<h4 style="color: #2c3e50; margin-top: 1.5rem;">Top 5 Medications Prescribed by Hospital</h4>', unsafe_allow_html=True)

                    for hospital_name, med_counts in medication_patterns['hospital_patterns'].items():
                        with st.expander(f"{hospital_name} - Top Medications"):
                            fig_hospital_med = px.bar(
                                x=med_counts.values,
                                y=med_counts.index,
                                orientation='h',
                                title=f'Top 5 Medications at {hospital_name}',
                                labels={'x': 'Number of Patients', 'y': 'Medication'},
                                color=med_counts.values,
                                color_continuous_scale='Reds'
                            )
                            st.plotly_chart(fig_hospital_med, use_container_width=True)

                # Patient burden analysis
                if 'patient_burden' in medication_patterns:
                    st.markdown(f'<div class="insight-box">{medication_patterns["patient_burden"]["analysis"]}</div>', unsafe_allow_html=True)

    # Key findings summary
    st.markdown('<p class="section-header">Key Healthcare Insights Summary</p>', unsafe_allow_html=True)

    key_findings = []

    # Patient distribution insight
    total_patients = len(data['patients']) if not data['patients'].empty else 0
    total_hospitals = len(data['hospitals']) if not data['hospitals'].empty else 0
    key_findings.append(f"Healthcare network serves {total_patients:,} patients across {total_hospitals} facilities")

    # Surgical insight
    if not data['surgeries'].empty:
        total_surgeries = len(data['surgeries'])
        unique_procedures = data['surgeries']['procedure_name'].nunique()
        key_findings.append(f"Surgical services performed {total_surgeries:,} procedures across {unique_procedures} different procedure types")

    # Medication insight
    if not data['drug_records'].empty:
        total_prescriptions = len(data['drug_records'])
        unique_patients_with_meds = data['drug_records']['patient_id'].nunique()
        medication_coverage = (unique_patients_with_meds / total_patients * 100) if total_patients > 0 else 0
        key_findings.append(f"Pharmaceutical services issued {total_prescriptions:,} prescriptions covering {medication_coverage:.1f}% of the patient population")

    # Monitoring insight
    if not data['vitals'].empty:
        total_vitals = len(data['vitals'])
        monitored_patients = data['vitals']['patient_id'].nunique()
        monitoring_coverage = (monitored_patients / total_patients * 100) if total_patients > 0 else 0
        avg_readings_per_patient = total_vitals / monitored_patients if monitored_patients > 0 else 0
        key_findings.append(f"Patient monitoring captured {total_vitals:,} vital sign readings from {monitoring_coverage:.1f}% of patients (average {avg_readings_per_patient:.1f} readings per monitored patient)")

    for finding in key_findings:
        st.markdown(f'<div class="pattern-card">{finding}</div>', unsafe_allow_html=True)

    # Data Quality Summary
    st.markdown('<p class="section-header">Data Summary</p>', unsafe_allow_html=True)

    data_summary = []
    for table_name, table_data in data.items():
        record_count = len(table_data) if not table_data.empty else 0
        data_summary.append({
            'Table': table_name.replace('_', ' ').title(),
            'Records': record_count
        })

    summary_df = pd.DataFrame(data_summary)
    st.dataframe(summary_df, use_container_width=True)

if __name__ == "__main__":
    main()
'''

# Save the complete geo-health dashboard
with open('geo_health_dashboard.py', 'w', encoding='utf-8') as f:
    f.write(dashboard_content)

print("✅ Complete Geo-Health Dashboard created successfully!")

# Deployment function
def deploy_complete_geo_dashboard():
    """Deploy the complete geo-health dashboard"""
    print("- DEPLOYING COMPLETE GEO-HEALTH DASHBOARD")
    print("=" * 50)

    # Validate data files
    import os

    base_paths = [
        "/content/drive/MyDrive/Xpertnurse Project/synthetic_medical_data/",
        "/content/drive/My Drive/Xpertnurse Project/synthetic_medical_data/",
        "/content/synthetic_medical_data/",
        "./"
    ]

    required_files = ["patients.csv", "vitals.csv", "doctors.csv", "hospitals.csv"]

    data_found = False
    for path in base_paths:
        if all(os.path.exists(path + file) for file in required_files):
            print(f"✅ Core data files found at: {path}")
            data_found = True
            break

    if not data_found:
        print("❌ Required data files not found")
        return None

    # Mount Google Drive if needed
    try:
        from google.colab import drive
        if not os.path.exists('/content/drive'):
            print("- Mounting Google Drive...")
            drive.mount('/content/drive')
        print("- Google Drive ready")
    except:
        print("- Running outside Colab - skipping drive mount")

    # Get ngrok token
    import getpass
    print("- Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken")
    token = getpass.getpass("Enter ngrok token (hidden): ")

    # Set up ngrok
    try:
        from pyngrok import ngrok
        ngrok.set_auth_token(token)
        print("✅ Ngrok authentication successful")
    except Exception as e:
        print(f"❌ Ngrok setup failed: {e}")
        return None

    # Start Streamlit
    import subprocess
    import time

    print("- Starting Complete Geo-Health Dashboard...")

    # Kill existing processes
    try:
        subprocess.run(['pkill', '-f', 'streamlit'], check=False)
        time.sleep(3)
    except:
        pass

    # Start new process
    process = subprocess.Popen([
        'streamlit', 'run', 'geo_health_dashboard.py',
        '--server.port', '8503',
        '--server.headless', 'true',
        '--server.enableCORS', 'false'
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    print("- Waiting for dashboard startup...")
    time.sleep(20)

    # Create tunnel
    try:
        ngrok.kill()  # Kill existing tunnels
        public_url = ngrok.connect(8503)

        print("- COMPLETE GEO-HEALTH DASHBOARD DEPLOYED!")
        print("=" * 50)
        print(f"🌐 Dashboard URL: {public_url}\n")

        print("- COMPLETE FEATURES:")
        print("- COMPLETE FEATURES:")
        print("  • Regional health pattern analysis with geographic clustering")
        print("  • Individual hospital performance metrics and mapping")
        print("  • Surgery search by procedure name with hospital-level counts")
        print("  • Medication prescription patterns (current vs lifetime)")
        print("  • Interactive filters (date range, regions, analysis components)")
        print("  • Comparative visualizations across regions and hospitals")
        print("  • Key healthcare insights and data quality summary\n")
        print(" READY FOR PRODUCTION USE")

        return public_url

    except Exception as e:
        print(f" Tunnel creation failed: {e}")
        return None

print("""
 COMPLETE GEO-HEALTH DASHBOARD - READY TO DEPLOY
=================================================

DEPLOYMENT:
Run: deploy_complete_geo_dashboard()

The dashboard is now complete with no truncation issues and
ready for production deployment in your Google Colab environment.
""")

Installing required packages...
✅ Complete Geo-Health Dashboard created successfully!

 COMPLETE GEO-HEALTH DASHBOARD - READY TO DEPLOY

DEPLOYMENT:
Run: deploy_complete_geo_dashboard()

The dashboard is now complete with no truncation issues and
ready for production deployment in your Google Colab environment.



## Run

In [ ]:
deploy_complete_geo_dashboard()

- DEPLOYING COMPLETE GEO-HEALTH DASHBOARD
✅ Core data files found at: /content/drive/MyDrive/Xpertnurse Project/synthetic_medical_data/
- Google Drive ready
- Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken
Enter ngrok token (hidden): ··········
✅ Ngrok authentication successful
- Starting Complete Geo-Health Dashboard...
- Waiting for dashboard startup...
- COMPLETE GEO-HEALTH DASHBOARD DEPLOYED!
🌐 Dashboard URL: NgrokTunnel: "https://1f2697182122.ngrok-free.app" -> "http://localhost:8503"

- COMPLETE FEATURES:
- COMPLETE FEATURES:
  • Regional health pattern analysis with geographic clustering
  • Individual hospital performance metrics and mapping
  • Surgery search by procedure name with hospital-level counts
  • Medication prescription patterns (current vs lifetime)
  • Interactive filters (date range, regions, analysis components)
  • Comparative visualizations across regions and hospitals
  • Key healthcare insights and data quality summary

 READY FO

<NgrokTunnel: "https://1f2697182122.ngrok-free.app" -> "http://localhost:8503">